# train 데이터셋의 context 데이터 탐색
1. 데이터 기본 정보 확인
2. 데이터 중복 확인 및 데이터 탐색
3. 언어의 종류 확인
4. 데이터 전처리 필요 부분 확인 (문법, 어휘, 특수 문자 등)

# 0. 데이터셋 및 탐색 준비

In [ ]:
# 데이터셋 부르기 위한 datasets 라이브러리 다운로드
!pip install datasets

In [2]:
# 탐색에 필요한 라이브러리 import
from datasets import load_from_disk
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from collections import Counter
import re

In [3]:
# pandas datasframe으로 변환 및 필요한 컬럼만 추출하는 함수
def dataset_to_df(dataset):
  data_list = []

  for data in dataset:
    data_list.append({
        'title' : data["title"],
        "context" : data["context"],
        "question" : data["question"],
        "answer_start" : data["answers"]["answer_start"][0],
        "answer_text" : data["answers"]["text"][0],
        "document_id" : data["document_id"]
    })
  df = pd.DataFrame(data_list)
  return df

In [4]:
# 데이터셋 불러오기
path = "/content/drive/MyDrive/네이버 부스트 캠프/프로젝트/Level2_프로젝트/MRC/data/train_dataset"
dataset = load_from_disk(path)
print(dataset)
print(dataset['train'])

DatasetDict({
    train: Dataset({
        features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
        num_rows: 3952
    })
    validation: Dataset({
        features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
        num_rows: 240
    })
})
Dataset({
    features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
    num_rows: 3952
})


1.

In [5]:
train_df = dataset_to_df(dataset["train"])
train_df.head()

,title,context,question,answer_start,answer_text,document_id
0,미국 상원,미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국...,대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?,235,하원,18293
1,인사조직관리,'근대적 경영학' 또는 '고전적 경영학'에서 현대적 경영학으로 전환되는 시기는 19...,현대적 인사조직관리의 시발점이 된 책은?,212,《경영의 실제》,51638
2,강희제,강희제는 강화된 황권으로 거의 황제 중심의 독단적으로 나라를 이끌어 갔기에 자칫 전...,강희제가 1717년에 쓴 글은 누구를 위해 쓰여졌는가?,510,백성,5028
3,금동삼존불감,"불상을 모시기 위해 나무나 돌, 쇠 등을 깎아 일반적인 건축물보다 작은 규모로 만든...",11~12세기에 제작된 본존불은 보통 어떤 나라의 특징이 전파되었나요?,625,중국,34146
4,계사명 사리구,동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기...,명문이 적힌 유물을 구성하는 그릇의 총 개수는?,30,4개,47334


# 1. 데이터 기본 정보 확인

In [6]:
train_df.head()

,title,context,question,answer_start,answer_text,document_id
0,미국 상원,미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국...,대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?,235,하원,18293
1,인사조직관리,'근대적 경영학' 또는 '고전적 경영학'에서 현대적 경영학으로 전환되는 시기는 19...,현대적 인사조직관리의 시발점이 된 책은?,212,《경영의 실제》,51638
2,강희제,강희제는 강화된 황권으로 거의 황제 중심의 독단적으로 나라를 이끌어 갔기에 자칫 전...,강희제가 1717년에 쓴 글은 누구를 위해 쓰여졌는가?,510,백성,5028
3,금동삼존불감,"불상을 모시기 위해 나무나 돌, 쇠 등을 깎아 일반적인 건축물보다 작은 규모로 만든...",11~12세기에 제작된 본존불은 보통 어떤 나라의 특징이 전파되었나요?,625,중국,34146
4,계사명 사리구,동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기...,명문이 적힌 유물을 구성하는 그릇의 총 개수는?,30,4개,47334


In [7]:
# 데이터의 개수
print(f"데이터 개수 : {len(train_df)}")

데이터 개수 : 3952


In [ ]:
# 랜덤 샘플링한 데이터
# 전체 데이터 중 200개를 랜덤으로 뽑기
sample_context = random.sample(list(train_df["context"]), 200)

print(sample_context)

In [ ]:
# 랜덤 샘플된 데이터 직접 확인하기
for context in sample_context:
  print(context)
  print("=" * 50)

- 확인된 전처리 필요한 문자 <br>
  - `\\n`, `\n`, `\n*`, `\\n\\n`, `\n\n`

- 확인된 언어의 종류<br>
  1. 한국어
  2. 영어
  3. 일본어
  4. 프랑스어
  5. 한자
  6. 아랍어

- 제거가 필요한지 애매한 문자<br>
  - `가우스 그룹#s-1.3`
  - 자세한 내용은 [confluence](https://) 참고

- 특이한 특수 문자
  - `△사용자`

# 1. 데이터 중복 확인 및 데이터 탐색


In [10]:
# 특정 칼럼을 기준으로 중복된 값을 Counter 함수로 찾은 뒤 리턴하는 함수
def count_duplication(column):
  count = dict(Counter(train_df[column]))
  count = sorted(count.items(),
                 reverse=True,
                 key=lambda item:item[1])

  return count

In [11]:
def print_duplication(column):
  total = len(train_df)
  unique_column = len(train_df[column].unique())

  print(f"전체 데이터 개수 : {total}")
  print(f"unique한 {column} 개수 : {unique_column}")

  if total != unique_column:
    print(f"중복된 값이 있습니다. 중복된 값 개수 : {total - unique_column}")
  else:
    print("중복된 값이 없습니다.")

## document_id 기준으로 중복 확인


In [12]:
# document_id 중복 여부
print_duplication("document_id")

전체 데이터 개수 : 3952
unique한 document_id 개수 : 3340
중복된 값이 있습니다. 중복된 값 개수 : 612


In [ ]:
document_count = count_duplication("document_id")
print(document_count)

In [14]:
train_df[train_df["document_id"] == document_count[0][0]]

,title,context,question,answer_start,answer_text,document_id
201,윤치호,"그러나 한편으로는 미국 사회의 인종차별주의적, 흑인을 차별하는 태도를 목격하면서 백...",평상시 기독교선교사들로부터 차별적 대우를 받은 윤치호가 생각한 미국의 중대한 속성은?,645,인종주의,5284
1426,윤치호,"그러나 한편으로는 미국 사회의 인종차별주의적, 흑인을 차별하는 태도를 목격하면서 백...",윤치호가 미국의 특징이 인종주의라고 생각하게 만든 인물은?,570,기독교선교사들,5284
1622,윤치호,"그러나 한편으로는 미국 사회의 인종차별주의적, 흑인을 차별하는 태도를 목격하면서 백...",조선에서도 윤치호를 유색 인종이라는 이유로 차별한 사람은?,254,미국인 선교사,5284
2199,윤치호,"그러나 한편으로는 미국 사회의 인종차별주의적, 흑인을 차별하는 태도를 목격하면서 백...",윤치호는 누구의 설교를 듣고 충격을 받았나?,824,남부 출신 남감리교회 목사들,5284


In [15]:
train_df[train_df["document_id"] == document_count[3][0]]

,title,context,question,answer_start,answer_text,document_id
110,빌 클린턴,클린턴의 첫 기간의 말기가 접근하면서 새로운 스캔들이 일어났다. 스캔들은 클린턴과 ...,클린턴의 두 번째 임기는 어떤 스캔들로 인해 내리막을 걷게 되었나?,257,화이트워터 사건,4633
3034,빌 클린턴,클린턴의 첫 기간의 말기가 접근하면서 새로운 스캔들이 일어났다. 스캔들은 클린턴과 ...,클린턴은 1996년 재선에서 누구를 꺾고 선거를 이겼나요?,188,자신의 공화당 상대 후보 밥 돌,4633
3818,빌 클린턴,클린턴의 첫 기간의 말기가 접근하면서 새로운 스캔들이 일어났다. 스캔들은 클린턴과 ...,두 번째 선거에서 빌 클린턴은 누구를 상대로 승리하였나?,202,밥 돌,4633


## title 기준으로 중복 확인

In [16]:
# title 중복 여부 확인
print_duplication("title")

전체 데이터 개수 : 3952
unique한 title 개수 : 2716
중복된 값이 있습니다. 중복된 값 개수 : 1236


In [ ]:
# title 중복 개수 확인
title_count = count_duplication("title")
print(title_count)

In [ ]:
train_df[train_df["title"] == title_count[0][0]]

In [19]:
train_df[(train_df["title"] == title_count[0][0]) & (train_df["answer_text"] == "다산 정약용")]

,title,context,question,answer_start,answer_text,document_id
19,윤치호,"혹자는 윤치호의 직계선조인 윤근수, 윤두수로 그의 문중은 당색으로는 서인 소론 계열...",윤치호가 극찬했던 조선시대의 인물은 누구인가요?,585,다산 정약용,5322
1212,윤치호,"혹자는 윤치호의 직계선조인 윤근수, 윤두수로 그의 문중은 당색으로는 서인 소론 계열...",천주교로 종교를 바꿨다는 의심을 받았던 학자는?,300,다산 정약용,5322
2442,윤치호,"혹자는 윤치호의 직계선조인 윤근수, 윤두수로 그의 문중은 당색으로는 서인 소론 계열...",윤치호가 조선에서 탄생한 위대한 인물이라 칭하며 극찬을 보낸 인물은?,585,다산 정약용,5322


## document_id가 같으면서 answer_start가 같은지 확인

In [ ]:
document_count = count_duplication("document_id")
print(document_count)

In [21]:
# document_id가 같으면서 answer_start가 같은 데이터 추출, document_id 값 추출
document_list = []
for count in document_count:
  if count[1] > 1:
    document_count_df = train_df[(train_df["document_id"] == count[0])]
    same_answer_start_df = document_count_df[document_count_df.duplicated('answer_start', keep=False)]
    if len(same_answer_start_df) > 0:
      document_list.append(count[0])

print(document_list)

[5322, 8428, 7623, 6653, 8190, 6716, 7139, 7833, 25068, 49393, 15663, 33391, 14917, 53065, 5031, 57931, 36795, 5282, 35777, 6711, 51137, 6907, 33784, 47393, 47169, 6329, 32475, 6541, 20027, 10686, 6773]


In [22]:
document_count_df = train_df[(train_df["document_id"] == 5322)]
same_answer_start_df = document_count_df[document_count_df.duplicated('answer_start', keep=False)]
same_answer_start_df

,title,context,question,answer_start,answer_text,document_id
19,윤치호,"혹자는 윤치호의 직계선조인 윤근수, 윤두수로 그의 문중은 당색으로는 서인 소론 계열...",윤치호가 극찬했던 조선시대의 인물은 누구인가요?,585,다산 정약용,5322
2442,윤치호,"혹자는 윤치호의 직계선조인 윤근수, 윤두수로 그의 문중은 당색으로는 서인 소론 계열...",윤치호가 조선에서 탄생한 위대한 인물이라 칭하며 극찬을 보낸 인물은?,585,다산 정약용,5322


In [23]:
document_answer_df = pd.DataFrame()
for document in document_list:
  document_count_df = train_df[(train_df["document_id"] == document)]
  same_answer_start_df = document_count_df[document_count_df.duplicated('answer_start', keep=False)]
  document_answer_df = pd.concat([document_answer_df, same_answer_start_df], ignore_index=True)

document_answer_df.head(10)

,title,context,question,answer_start,answer_text,document_id
0,윤치호,"혹자는 윤치호의 직계선조인 윤근수, 윤두수로 그의 문중은 당색으로는 서인 소론 계열...",윤치호가 극찬했던 조선시대의 인물은 누구인가요?,585,다산 정약용,5322
1,윤치호,"혹자는 윤치호의 직계선조인 윤근수, 윤두수로 그의 문중은 당색으로는 서인 소론 계열...",윤치호가 조선에서 탄생한 위대한 인물이라 칭하며 극찬을 보낸 인물은?,585,다산 정약용,5322
2,선언형 프로그래밍,"선언형 프로그래밍은 두 가지 뜻으로 통용되고 있다.\n\n한 정의에 따르면, 프로그...",선언형 프로그램과 대비되는 프로그램은?,328,명령형 프로그램,8428
3,선언형 프로그래밍,"선언형 프로그래밍은 두 가지 뜻으로 통용되고 있다.\n\n한 정의에 따르면, 프로그...",선언형 프로그램의 알고리즘에 대한 정의와 반대인 개념은?,328,명령형 프로그램,8428
4,꿀풀과,"꿀풀과(--科, 라미아케아이)는 꿀풀목의 과이다. 꿀풀목 가운데 가장 큰 과이며 전...",꿀풀목 가운데 가장 큰 과와 생김새가 비슷한 과는?,562,현삼과,7623
5,꿀풀과,"꿀풀과(--科, 라미아케아이)는 꿀풀목의 과이다. 꿀풀목 가운데 가장 큰 과이며 전...",꿀풀과와 유사하게 생긴 식물은?,562,현삼과,7623
6,꿀풀과,"꿀풀과(--科, 라미아케아이)는 꿀풀목의 과이다. 꿀풀목 가운데 가장 큰 과이며 전...",분과에 한 개의 씨가 들어있지 않은 풀의 종류는?,562,현삼,7623
7,모아이,"모아이(Moai, Moái, 모이아스)는 칠레 이스터 섬에 있는 사람 얼굴 모양의 ...",모아이는 무엇으로 이루어져있나?,445,화산암,6653
8,모아이,"모아이(Moai, Moái, 모이아스)는 칠레 이스터 섬에 있는 사람 얼굴 모양의 ...",모아이를 이루는 돌의 종류는 무엇인가?,445,화산암,6653
9,베트남의 역사,베트남 전쟁\n\n 베트남 전쟁은 1964년 통킹만 사건을 구실로 미국이 북 베트남...,제2차 인도차이나 전쟁이 끝났던 해는?,131,1975년,8190


# 3. 언어의 종류 확인

In [24]:
language_train_df = train_df
language_train_df.head()

,title,context,question,answer_start,answer_text,document_id
0,미국 상원,미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국...,대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?,235,하원,18293
1,인사조직관리,'근대적 경영학' 또는 '고전적 경영학'에서 현대적 경영학으로 전환되는 시기는 19...,현대적 인사조직관리의 시발점이 된 책은?,212,《경영의 실제》,51638
2,강희제,강희제는 강화된 황권으로 거의 황제 중심의 독단적으로 나라를 이끌어 갔기에 자칫 전...,강희제가 1717년에 쓴 글은 누구를 위해 쓰여졌는가?,510,백성,5028
3,금동삼존불감,"불상을 모시기 위해 나무나 돌, 쇠 등을 깎아 일반적인 건축물보다 작은 규모로 만든...",11~12세기에 제작된 본존불은 보통 어떤 나라의 특징이 전파되었나요?,625,중국,34146
4,계사명 사리구,동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기...,명문이 적힌 유물을 구성하는 그릇의 총 개수는?,30,4개,47334


In [25]:
# 언어 분류 함수
def classify_language(text):
    has_english = False
    has_korean = False
    has_japanese = False
    has_chinese = False
    has_arabic = False
    has_other = False

    for char in text:
        # 영어 (A-Z, a-z)
        if 'A' <= char <= 'Z' or 'a' <= char <= 'z':
            has_english = True
        # 한글 (유니코드 범위: U+AC00 ~ U+D7A3)
        elif '가' <= char <= '힣':
            has_korean = True
        # 일본어 히라가나 (U+3040 ~ U+309F) 및 가타카나 (U+30A0 ~ U+30FF)
        elif '\u3040' <= char <= '\u309F' or '\u30A0' <= char <= '\u30FF':
            has_japanese = True
        # 한자 (유니코드 범위: U+4E00 ~ U+9FFF)
        elif '\u4E00' <= char <= '\u9FFF':
            has_chinese = True
        # 아랍어 (유니코드 범위: U+0600 ~ U+06FF)
        elif '\u0600' <= char <= '\u06FF':
            has_arabic = True
        # 기타 문자
        else:
            has_other = True

    # 결과 반환
    languages = []
    if has_korean:
        languages.append("Korean")
    if has_english:
        languages.append("English")
    if has_japanese:
        languages.append("Japanese")
    if has_chinese:
        languages.append("Chinese")
    if has_arabic:
        languages.append("Arabic")
    if has_other and not languages:
        languages.append("Other")

    return ", ".join(languages) if languages else "Other"

# 'context' 열에서 언어 분석 후 새로운 열 추가
language_train_df['languages'] = language_train_df['context'].apply(classify_language)

# 결과 출력
print(language_train_df[['context', 'languages']])

                                                context  \
0     미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국...   
1     '근대적 경영학' 또는 '고전적 경영학'에서 현대적 경영학으로 전환되는 시기는 19...   
2     강희제는 강화된 황권으로 거의 황제 중심의 독단적으로 나라를 이끌어 갔기에 자칫 전...   
3     불상을 모시기 위해 나무나 돌, 쇠 등을 깎아 일반적인 건축물보다 작은 규모로 만든...   
4     동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기...   
...                                                 ...   
3947  이오의 산\n이오의 산 목록\n 이오에는 100~150개의 산이 있다. 이들 산의 ...   
3948  애니의 고군분투 뉴욕 입성기!!\n\n인류학자가 꿈인 21살 소녀 '애니(스칼렛 요...   
3949  1842년에 작곡가이자 지휘자인 오토 니콜라이가 빈 궁정 오페라극장 소속 관현악단을...   
3950  원어는 고대 그리스어까지 거슬러 올라간다. 영어 문헌에 이 말이 나타나기 시작한 것...   
3951  2008년 2월 28일 실시된 2008년 함부르크 주의회 선거에서 기민련은 과반수 ...   

                     languages  
0              Korean, English  
1              Korean, English  
2     Korean, English, Chinese  
3     Korean, English, Chinese  
4     Korean, English, Chinese  
...                        ...  
3947  Korean, English, Chinese  
3948           Korean, Engl

In [26]:
language_train_df.head(10)

,title,context,question,answer_start,answer_text,document_id,languages
0,미국 상원,미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국...,대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?,235,하원,18293,"Korean, English"
1,인사조직관리,'근대적 경영학' 또는 '고전적 경영학'에서 현대적 경영학으로 전환되는 시기는 19...,현대적 인사조직관리의 시발점이 된 책은?,212,《경영의 실제》,51638,"Korean, English"
2,강희제,강희제는 강화된 황권으로 거의 황제 중심의 독단적으로 나라를 이끌어 갔기에 자칫 전...,강희제가 1717년에 쓴 글은 누구를 위해 쓰여졌는가?,510,백성,5028,"Korean, English, Chinese"
3,금동삼존불감,"불상을 모시기 위해 나무나 돌, 쇠 등을 깎아 일반적인 건축물보다 작은 규모로 만든...",11~12세기에 제작된 본존불은 보통 어떤 나라의 특징이 전파되었나요?,625,중국,34146,"Korean, English, Chinese"
4,계사명 사리구,동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기...,명문이 적힌 유물을 구성하는 그릇의 총 개수는?,30,4개,47334,"Korean, English, Chinese"
5,용아병,델포이의 신탁에 따라 암소를 따라간 카드모스는 테베 땅에 이르렀다. 카드모스는 암소...,카드모스의 부하들이 간 곳에는 무엇이 있었는가?,91,드래곤,37482,"Korean, English"
6,호반,"제27회와 제73회에 출현한다. 관우는 자신을 저지하는 공수, 맹탄, 한복, 변희를...",관우를 불태워 죽이려한 사람 누구인가?,68,형양태수 왕식,24395,"Korean, English, Chinese"
7,제50노섬브리안 보병사단,이집트로 이어지는 해안 도로는 1개의 사단만이 방어할 수 있었으며 엘 아뎀 요새와 ...,참호 속에 무기와 장비를 버리고 도주한 집단은?,583,이탈리아군,43399,"Korean, English"
8,오싱,하지만 슬픈 일들이 그녀의 행복을 질시하고 있었다.\n평생 고생만 한 어머니를 백혈...,제2차 세계 대전에 참전하여 사망한 자식은?,195,큰아들 유,10426,"Korean, English, Chinese"
9,모후산,모후산(母后山)대한민국 전라남도 화순군과 순천시의 경계를 이루는 높이 943.7m의...,고려 공민왕이 처가 식구들과 아내와 함께 피신처로 삼은 마을은?,861,왕대마을,15766,"Korean, English, Chinese"


In [27]:
# context 데이터에 사용된 언어 종류 확인
unique_languages = set()

for language in list(language_train_df["languages"].unique()):
    languages = language.split(', ')
    unique_languages.update(languages)

result = list(unique_languages)

print(result)

['Korean', 'Arabic', 'English', 'Chinese', 'Japanese']


In [ ]:
contains_df = language_train_df[language_train_df["context"].str.contains("폴 르 노르망")]
print(contains_df.to_string())

# 4.데이터 전처리 필요 부분 확인 (문법, 어휘, 특수 문자 등)


In [29]:
# 개행 문자 카운트 함수 (by 정휘님)
def get_newline(data_samples):
    all_lines = []
    for i,data in enumerate(data_samples):
        new_line_count = data.count("\\n")
        all_lines.append([i, new_line_count])
    return all_lines

In [30]:
newline_counts = np.array(get_newline(train_df["context"]))
print(np.sum(newline_counts[:,1],axis=0))

22808


In [31]:
# 특수 문자를 찾는 함수 정의
def find_special_characters(text, pattern):
    # 정규 표현식을 사용하여 특수 문자 찾기
    return re.findall(pattern, text)

# context에서 모든 특수 문자를 찾고 중복 제거
all_special_chars = set()
#pattern = r'[^a-zA-Z0-9가-힣\s]' # 알파벳, 숫자, 한글, 공백 제외
#pattern = r'[^a-zA-Z0-9가-힣\s\u4e00-\u9fff]' # 알파벳, 숫자, 한글, 한자 공백 제외
pattern = r'[^a-zA-Z0-9가-힣\s\u4e00-\u9FFF\u3040-\u309F\u30A0-\u30FF\u0600-\u06FF]' # 알파벳, 숫자, 한글, 한자, 일본어, 아랍어 공백 제외
for context in train_df['context']:
    special_chars = find_special_characters(context, pattern)
    all_special_chars.update(special_chars)

# 결과 출력
print(f"찾은 특수 문자 종류: {all_special_chars}")
print(f"특수 문자의 개수: {len(all_special_chars)}")

찾은 특수 문자 종류: {'া', 'υ', 'о', 'θ', '°', 'Н', '③', '＞', 'Þ', 'µ', 'η', 'н', 'ழ', 'е', 'ü', 'क', '“', '\u200e', 'ㆍ', '+', '『', '⑦', 'р', 'ό', 'ɔ', 'ׇ', 'ா', '}', 'ㅋ', '→', 'ˈ', 'І', '△', 'ர', 'ㅁ', '±', 'Α', 'ნ', 'ː', '㎞', 'ㄷ', 'ᛁ', 'μ', ',', '←', 'ე', '３', 'ă', '〕', 'α', 'ი', '】', '。', 'ŭ', '㎜', '＼', '"', '·', '１', 'в', 'ν', ':', 'ि', 'י', 'ὔ', '…', 'Ｖ', 'π', 'ə', '―', '’', '(', '☆', 'ờ', '（', 'é', '▲', 'Κ', '&', 'з', ']', '□', "'", 'ᛚ', 'с', 'Φ', 'ा', 'ú', 'ய', '\xad', '⑪', '\uf000', 'გ', 'त', 'ㅅ', 'დ', 'ს', 'м', '○', '*', '#', 'Ｍ', 'ㄹ', 'к', '８', 'á', '５', 'æ', 'ê', 'ㅕ', '/', 'ㅏ', 'ć', '≫', 'ί', 'ǫ', 'ξ', 'ვ', 'ý', 'ɛ', 'γ', 'ð', ')', 'Р', '.', '㎠', 'ε', 'ㅚ', '～', '↓', '♣', 'ო', 'Δ', 'ώ', 'í', 'É', '？', 'ㄱ', 'ㅎ', 'ხ', 'ஜ', 'ც', '∧', 'ó', '_', '⑤', 'й', 'Ε', 'ơ', '⑥', '²', 'х', '—', '－', 'ω', 'ë', 'В', '%', '$', '்', '`', 'ē', '́', 'ल', '㎝', 'ַ', '４', 'έ', '」', 'ч', 'і', 'ì', 'à', 'β', 'Ｘ', 'Ἰ', 'л', 'Λ', 'ש', 'χ', 'Π', 'ோ', '×', 'κ', '?', 'ㄴ', '℃', '|', 'ğ', 'œ', '【', '〔', 'ά', '¾', 'স'

In [32]:
pd.set_option("display.max_colwidth", None)

In [33]:
train_df[train_df["context"].str.contains("\uf000")]["context"]

,context
961,"구포(龜浦)라는 지명은 거북龜자로 시작되므로 거북이에 대한 이야기가 전해 내려오고 있다. 그 중에서 《양산군지(梁山郡誌)》에 나오는 구포에 관한 기록은 “창(倉)“을 설치하였으므로 남창(南倉)이라고 하며 범방산(泛舫山) 한 줄기가 낙동강을 향하여 머리에 돌을 이고 있는 모습이 거북이와 같다.는 연유에서 구포 지명의 유래를 밝히고 있다.\n\n여기에 나오는 범방산은 백양산(운수산) 줄기로서 강변쪽으로 뻗어내린 구포의 남쪽 구남마을 뒷산을 말한다. 현재 모라동 쪽에서 구포도서관, 백양고, 구포중, 모라중이 들어 서 있는 곳으로 뻗어내려 구포초등학교까지 이어지는 산이다.\n\n이 산의 형상을 강 건너쪽 멀리서 바라보면 거북이의 머리와 몸체를 그대로 빼어 놓은 것처럼 닮아 보인다. 범방산 능선에는 온갖 바위들이 소나무와 얽혀 군집해 있는데 《양산군지》의 기록처럼 머리에 돌을 이고 있는 듯한 모습을 보이고 있는 것이다. 그리고 범방산을 구포 본 동네에서 쳐다보면 산등성이에 거북이 형상을 한 큰 바위가 산으로 오르는 모습을 하고 있다.\n\n이처럼 거북이 형상을 한 산줄기에 거북바위까지 있어 구포사람들은 이 산을 거북산이라고 부르는데 구포의 지명이 여기에서 왔다는 것이다. 옛날 바다의 거북이가 겨울에도 따뜻한 기운이 있어 이를 보고 사람들이 바다의 거북이가 물가 모래밭에 구멍을 파고 겨울잠을 자면서 쉬어가는 곳이다 하여 구포의 지명에 유래를 들고 있다. 그래서 거북이가 쉬어가는 갯가로서 ""거부개""즉 ""龜浦""라는 것이다.\n\n조선시대 구포를 감동진, 감동나루로 불러왔는데 이 감동진나루는 낙동강 3대 나루터의 한 곳으로 수로교통과 교역의 시발지로서 강변 언덕위에는 정부의 세곡(稅穀)을 보관하는 남창(南倉)이 설치되어 있었다. 그래서 구포 감동나루가 있는 이곳의 행정지명이 양산군 좌이면\n남창리로 나와 있다.\n\n남창을 배경으로 강변에서부터 안쪽 넓은 공터에서 구포장(場)이 섰는데 장타령에 나올만큼 큰 장터였고 곡물선과 상선, 어선들이 드나들면서 크게 번창했는데 근세에 이르기까지 낙동강 수로교통의 시발지로서 구포는 널리 알려져 왔다."


In [34]:
special_char = "⑥"
train_df[train_df["context"].str.contains(special_char)]["context"]

,context
308,"구매품의 수량 결정은 연속생산을 위한 상비품인가, 주문생산을 위한 주문품인가에 따라 다르다. 즉 주문품의 경우 요구량을 기준으로 하지만, 상비품의 경우는 주문량에 의한 단가나 단위당 구매비용이 저렴하고, 저장비용을 늘여서 자금고정을 크게 하는 것을 고려, 구매 및 저장에 따르는 원가를 우선 싸게 계획해야 한다. 구매시기는 수량에 따라 결정된다.\n구매수량 및 시기결정에 있어 고려해야 할 요소는 다음과 같다. ① 총재고량 및 유효재고량, ② 사용량은 계속사용·임시사용·계절적 사용을 고려한다. ③ 발주에서 납품까지에 소요되는 기간, ④ 납기지연에 대비하는 예비량, ⑤ 수요변동에 대비하는 긴급수량, ⑥ 자재보유기간, ⑦ 대량매입으로 유리하게 되는 양, ⑧ 투입자금과 그 효율, ⑨ 기타 자재획득에 소요되는 비용, 예를 들면 운임·수수료·보험료 등이다.\n〔그림 2〕는 구매수량 및 시기와 재고량·사용량 등의 상호관계를 나타내는데, 이 그림에 의해 최대·최소 재고량, 평균사용량 및 조달기간이 주어지면 적정주문량·발주시기가 결정된다.\n또한 구매에 있어 단위당 '주문비용'과 '저장비용'이 같아지기 때문에 다음과 같은 관계가 성립되며, 이로써 경제적 주문량()이 구해진다.\n：단위당 주문비용 ：단위당 저장비용\n：경제적 주문량 ：1단위당 원가\n：재고비용(이자·보험료 등의 율)\n：연 구매량(또는 사용량)"
1018,"일반 상대성 이론\n\n일반 상대성 이론은 1915년에 아인슈타인이 발표하였다. (프로이센 과학 아카데미에서 1915년 11월 25일에 제출) 일반 상대성이론은 특수 상대성이론의 확장판이라 하면 이해하기 쉬울 것이다. 이 이론은 뉴턴의 고전 물리학에 결정타를 날림으로서 새로운 물리학적 이론의 길을 열었다는 점에서 의의가 있다고 할 수 있다.\n\n일반상대성이론에 대해 알아보기 전에 먼저 이를 전개하기 위해 필요한 한 가지 가정을 보도록 하자.\n\n등가원리\n-가속 좌표계에서 지구로 인해 생기는 중력과 중력가속도g와 같은 크기의 가속도 a로 중력의 작용방향과 반대로 운동하는 것으로 인한 관성력은 구분할 수 없다.\n\n즉, 관성질량과 중력질량이 같은 측정값을 지닌다는 것이다.\n\n1. 시공간\n일반 상대성 이론에서 아인슈타인은 '시공간은 4차원, 즉 시간과 공간이 결합된 형태의 연속체'임을 규명했다. 뉴턴이 시공간이 시간과 공간으로 분리되어 서로 영향을 주고받지 않는 별개의 공간인 점과, 공간위의 한 점에 위치한 물체에 어떠한 영향도 받지 않을 것이라고 생각한 반면에 아인슈타인의 사고실험에서 아인슈타인은 시공간이 서로 상호작용함과, 공간위에 위치한 어떠한 질량을 가진 물체에 대하여 공간이 휘어짐을 제시하였다.\n\n2. 중력\n아인슈타인은 뉴턴의 중력을 가속운동계에 적용시켰다. 그는 중력가속도g의 크기만큼 가속하는 가속계 내의 물체와, 중력을 받고있는 물체는 서로 구분이 불가능하다는 이론을 내세움으로서 가속계를 관성계로 해석가능함을 보였다.\n\n3. 중력으로 인한 시간팽창\n위에서 가속계 역시 관성계로 인식할 수 있다고 하였다. 따라서 가속계의 물체는 관성계에서 물체의 이동으로 판단할 수 있고, 특수 상대성이론에 따라 시간지연이 일어남을 알 수 있다. 이로서 중력을 받는 물체는 그 물체에 흐르는 시간이 느려지게 된다.\n\n4. 공간 왜곡으로 일어나는 현상\n4-1. 빛의 휘어짐 : 빛의 움직임을 가속운동계에 적용시켜보면, 그 빛이 휜다는 것을 알 수 있을 것이다. 따라서 중력으로 인하여 빛이 휜다는 것을 알 수 있다\n\n4-2. 중력렌즈효과 : 이는 일식이 일어날 때 관측할 수 있는 현상인데, 바로 태양의 뒤에 위치하여 가려져 있던 천체가 태양의 중력에 의한 영향으로 그 빛이 휘어져 우리눈에 들어오는 것이다. 흔히 '아인슈타인의 십자가'라는 것이 이로 인한 현상이다.\n\n4-3. 블랙홀 : 질량이 매우 큰 천체는 공간을 심하게 왜곡하여 빛마저 삼켜버릴 수 있다.\n\n5. 일반상대성이론의 증거\n① 에딩턴의 태양의 일식 관측 : 지구에서 별을 관측할 때, 별과 지구사이에 태양이 있을 때와 없을 때의 별의 관측위치를 비교하여 빛이 휘는 것을 관찰\n\n② 수성의 세차운동 : 뉴턴의 이론에 따라 계산하면 100년동안 574´´만큼 이동해야 하지만, 실제 세차운동 관측결과 43´´정도 오차가 났다. 하지만 일반상대성 이론에 시공간의 곡률을 고려한 결과 이 차이를 정확히 설명\n\n③ 중력렌즈 : 중력이 렌즈처럼 빛을 휘게 하는 현상이다. 대표적으로 퀘이사가 있다.(퀘이사는 은하의 중력 때문에 지구에서 4개의 빛나는 쌍둥이별로 오인했었다.)\n\n④ 중력파 : 천체의 중력붕괴나 초신성폭발 같은 우주현상으로 발생하여 시공간이 일그러짐이 광속으로 파도처럼 전달되는 것을 말한다.(이전까지 발견되지 않고있다가 최근 블랙홀에서 발견되었다.)\n\n⑤ GPS : 인공위성이 움직이기 때문에 시간차이가 생기는데 속력과 중력의 작용을 고려하여 보정해줘야 한다.\n\n⑥ 블랙홀 : 질량이 매우 큰 천체는 공간을 휘게하여 천체를 지나는 빛마저 흡수한다.(아인슈타인이 2차원의 평면에 시간의 곡률을 표기한 그림에서 보면 블랙홀은 질량이 극도로 커 평면 자체가 엄청나게 움푹 들어가있다. 이때 블랙홀의 부분중 넘게되면 빛조차 절대 빠져나올수 없는 선이있는데 이곳을 사건의 지평선이라고 하며 이 안쪽부분은 어떠한것도 빠져나오지 못하므로 검게보인다.)"
3805,"일반 상대성 이론\n\n일반 상대성 이론은 1915년에 아인슈타인이 발표하였다. (프로이센 과학 아카데미에서 1915년 11월 25일에 제출) 일반 상대성이론은 특수 상대성이론의 확장판이라 하면 이해하기 쉬울 것이다. 이 이론은 뉴턴의 고전 물리학에 결정타를 날림으로서 새로운 물리학적 이론의 길을 열었다는 점에서 의의가 있다고 할 수 있다.\n\n일반상대성이론에 대해 알아보기 전에 먼저 이를 전개하기 위해 필요한 한 가지 가정을 보도록 하자.\n\n등가원리\n-가속 좌표계에서 지구로 인해 생기는 중력과 중력가속도g와 같은 크기의 가속도 a로 중력의 작용방향과 반대로 운동하는 것으로 인한 관성력은 구분할 수 없다.\n\n즉, 관성질량과 중력질량이 같은 측정값을 지닌다는 것이다.\n\n1. 시공간\n일반 상대성 이론에서 아인슈타인은 '시공간은 4차원, 즉 시간과 공간이 결합된 형태의 연속체'임을 규명했다. 뉴턴이 시공간이 시간과 공간으로 분리되어 서로 영향을 주고받지 않는 별개의 공간인 점과, 공간위의 한 점에 위치한 물체에 어떠한 영향도 받지 않을 것이라고 생각한 반면에 아인슈타인의 사고실험에서 아인슈타인은 시공간이 서로 상호작용함과, 공간위에 위치한 어떠한 질량을 가진 물체에 대하여 공간이 휘어짐을 제시하였다.\n\n2. 중력\n아인슈타인은 뉴턴의 중력을 가속운동계에 적용시켰다. 그는 중력가속도g의 크기만큼 가속하는 가속계 내의 물체와, 중력을 받고있는 물체는 서로 구분이 불가능하다는 이론을 내세움으로서 가속계를 관성계로 해석가능함을 보였다.\n\n3. 중력으로 인한 시간팽창\n위에서 가속계 역시 관성계로 인식할 수 있다고 하였다. 따라서 가속계의 물체는 관성계에서 물체의 이동으로 판단할 수 있고, 특수 상대성이론에 따라 시간지연이 일어남을 알 수 있다. 이로서 중력을 받는 물체는 그 물체에 흐르는 시간이 느려지게 된다.\n\n4. 공간 왜곡으로 일어나는 현상\n4-1. 빛의 휘어짐 : 빛의 움직임을 가속운동계에 적용시켜보면, 그 빛이 휜다는 것을 알 수 있을 것이다. 따라서 중력으로 인하여 빛이 휜다는 것을 알 수 있다\n\n4-2. 중력렌즈효과 : 이는 일식이 일어날 때 관측할 수 있는 현상인데, 바로 태양의 뒤에 위치하여 가려져 있던 천체가 태양의 중력에 의한 영향으로 그 빛이 휘어져 우리눈에 들어오는 것이다. 흔히 '아인슈타인의 십자가'라는 것이 이로 인한 현상이다.\n\n4-3. 블랙홀 : 질량이 매우 큰 천체는 공간을 심하게 왜곡하여 빛마저 삼켜버릴 수 있다.\n\n5. 일반상대성이론의 증거\n① 에딩턴의 태양의 일식 관측 : 지구에서 별을 관측할 때, 별과 지구사이에 태양이 있을 때와 없을 때의 별의 관측위치를 비교하여 빛이 휘는 것을 관찰\n\n② 수성의

In [35]:
special_char = "ä"
train_df[train_df["context"].str.contains(special_char)]["context"]

,context
1144,"아른험에 도착하자 사단의 재편성이 시작되었다. 잔존한 기갑 차량 대부분이 독일 본토의 정비공장으로 운송되기 위해 기차에 선적되었다. 9월 17일 일요일, 연합군의 마켓가든 작전이 개시되었고 영국 제1 공수 사단이 아른험 서쪽 오스테르베이크(Oosterbeek)에 강하했다. 위험을 감지한 제2 SS 기갑 군단장 비트리히 SS대장은 호엔슈타우펜과 프룬츠베르크에 임전태세를 취하라는 명령을 하달했다. 사단의 기갑 차량들이 기차에서 하역되었고, 정비 부대는 기갑 차량들의 궤도간격을 재조정하기 위해 미친듯이 작업했다. 사단 기갑 부대 중 오직 정찰 부대인 제9 SS 기갑수색 대대만이 구륜 및 반궤도 차량을 정수에 가깝게 보유하고 있었기에 즉시 투입이 가능했다.\n\n비트리히 SS대장은 호엔슈타우펜에 아른험을 점거하고 중요 지물인 아른험 교량을 확보하라는 명령을 하달했다. 하르처 SS중령은 사단을 아른험에 투입했고, 그들이 로텐 토이펠(Roten Teufel = 붉은 악마)이라 부른 영군 강하병들의 강력한 저항에 부딪혔다. 파울 그레브너(Paul Gräbner) SS대위가 지휘하는 기갑수색 대대가 네이메헌(Nijmegen) 부근으로 정찰을 목적으로 투입되었다. 그레브너 SS대위는 노르망디에서의 활약으로 당일 기사십자장을 수여받았다.\n\n기갑수색 대대가 아른험 남쪽에서 정찰을 하고 있을 무렵, 존 프로스트(John Frost) 중령의 영국 제1 공수 여단 2대대가 아른험으로 진입했고 다리 북쪽 끝에 방어 진지를 구축했다. 9월 18일 아침, 그레브너는 남쪽에서 정찰 임무를 마치고 복귀해 아른험 다리를 확보하라는 명령을 하달받았다.\n\n그레브너의 무리한 계획은 수수께끼로 남아있지만, 확실한 건 그레브너가 다리를 탈환하거나 혹은 영국군의 방어선을 돌파해 아른험 수비에 투입된 사단 본대를 지원하려고 했다는 것이다. 어쨌든지 간에 그레브너 SS대위의 공격 시도는 처참하게 끝이 났다. 태세를 갖춘 영군 강하병들은 선두 차량 4량을 무사히 보낸 후 대전차 화기인 피아트 대전차 로켓, 화염방사기 그리고 소화기들로 공격을 개시했다. 2시간의 전투로 호엔슈타우펜 기갑수색 대대는 실질적으로 괴멸되었으며, 그로 인해 그레브너 SS대위를 포함해 약 70명이 전사했고 차량 22량이 파괴되었다. 머나먼 다리에 이 장면이 묘사되어 있다.\n\n8일에 걸쳐 벌어진 전투에 사단은 하루도 빠짐없이 참가했으며, 프로스트 중령의 대대와 아른험 서쪽에서 전투를 치렀다. 또한 오스테르베이크 부근에서 포위된 제1 공수 사단 본대를 지속적으로 압박했다. 아른험 전투는 호엔슈타우펜의 대승이었다. 여타 독일군 부대들의 지원을 바탕으로 피폐해진 사단이 정예 영국 공수 부대를 괴멸시킨 것이다. 격렬한 전투에도 불구하고 호엔슈타우펜 및 프룬츠베르크 장병들은 포로가 된 강하병들을 정중하게 대해주었으며, 비트리히 SS대장은 소련군조차도 능가하는 붉은 악마의 끈기와 전투력에 주목했다.\n\n마켓가든 작전의 패인이 ""하필이면 아른험에 주둔한"" 제9 SS기갑 사단 및 제10 SS기갑 사단 때문이라는 것이 기존 학설이었고, 영화 머나먼 다리 역시 코넬리우스 라이언의 동명 저서도 이 학설에 기초한 것이었다. 그러나 90년대 이후 새롭게 제기된 학설은 당시 제9 SS기갑 사단 및 제10 SS기갑 사단의 전력은 이름만 기갑사단일 뿐 전차는 거의 보유하지 않았으며, 앞서 언급된 그레브너 SS대위의 기갑수색 대대가 가용한 기갑 전력의 거의 전부였다는 사실이 밝혀졌다. 아울러 이 무렵 두 사단의 병력은 합쳐도 1개 사단에 미치지 못하는 수준이었다는 것이다. 아울러 마켓가든 작전의 실패 책임을 이 두 사단으로 돌려서 영국군이 저지른 실책과 제1 공수 사단의 사실상 전멸에 대한 책임을 모면하려는 것이라고 주장하기도 한다."
1536,"불을 가지고 노는 소녀는 리스벳이 1년동안의 세계여행을 끝내고 스웨덴으로 돌아오는 것으로 시작한다. 귀국한지 얼마 되지 않았을때 리스벳은 자신의 후견인인 비여만과 블룸크비스트의 동료 두명의 유력한 살인용의자로 몰린다. 이것은 소비에트 연방의 옛 스파이이자 리스벳의 친부인 알렉산더 잘라첸코(Alexander Zalachenko)와 스웨덴 비밀경찰인 세포(Säpo)간부 몇몇과의 음모였다. 세포의 간부몇몇은 잘라첸코의 소비에트 연방에 대한 배신 이후 그를 지켜주는 임무를 받았었는데, 그들은 잘라첸코의 모든 악행위를 덮어주는 역할을 해왔었다.\n\n잘라첸코는 러시아 총정보국에서 높은 직위를 갖고있는 군인이었는데, 그가 소비에트 연방을 배신했을때 세포가 그것을 주의깊게 보고 그의 불법적 행위들을 다 덮어주며 그에게로 부터 소비에트 연방에 대한 정보를 조금씩 받았다.\n잘라첸코는 아들이 하나 있었는데, 그는 리스벳의 이복형제로, 로날드 니덜만(Ronald Neidermann)이다. 그는 잘라첸코의 성매매에 대해 조사하던 블룸크비스트의 두 동료를 살해하고, 리스벳에게 복수를 하려는 비여만을 살해했다. 그리고 잘라첸코의 모든 만행을 덮기위해 모든 일을 리스벳이 누명을 쓰게 만들었다.\n\n블룸크비스트는 리스벳을 도와주려 하지만, 리스벳은 블룸크비스트가 자신의 인생에 다시 연관되는 것을 원하지 않았다. 소설의 끝자락에선 블룸크비스트가 중상을 입은 리스벳을 구하기 위해 잘라첸코의 농장으로 가는데 그곳에서 도망가고 있는 니덜만과 리스벳으로 인해 얼굴과 다리를 도끼로 찍힌 후 죽어가고 있는 피투성이의 잘라첸코를 발견한다. 니덜만을 총으로 위협하며 포박하고 잘라첸코의 집에 의식을 잃은 리스벳을 본 블룸크비스트는 즉시 구급차를 부른다.\n\n라르손의 두 번째 소설에선 리스벳의 유년기를 묘사하였다. 리스벳은 천재적인 지능을 갖고있지만 사회에 적응하지 못하며 자신을 해하려는 사람에게는 무차별적으로 폭력적이라고 하였다. 이 모든 성격은 불우한 집안문제로 인한 결과였다. 잘라첸코는 반복적으로 리스벳의 어머니를 신체적과 정신적으로 폭력을 가했지만 스웨덴 정부는 잘라첸코의 존재가 리스벳의 엄마의 존재보다 더 중요하기 때문에 아무런 조치를 취하지 않았다. 잘라첸코는 리스벳의 쌍둥이 자매인 카밀라(Camilla)와 리스벳의 관계또한 망쳐놓았는데, 이는 카밀라는 잘라첸코를 상냥하고 점잖은 사람으로 기억하려 했기 때문이다. 예를 들어 잘라첸코가 방에서 문을 잠그고 리스벳의 어머니를 폭행한 후 나왔을때 카밀라는 잘라첸코에게 미움받지 않기위해 웃으며 안기기를 반복했기 때문이다.\n\n그러던 어느날, 리스벳이 12살 일때 잘라첸코는 리스벳의 어머니를 너무 심하게 폭행한 나머지 뇌손상을 입혔다. 그동안 아버지에 대한 복수를 꿈꾸던 리스벳은 휘발류를 모아왔는데, 그날 리스벳은 차에 탑승한 잘라첸코에게 휘발유를 들이부은 후 불붙은 성냥을 그에게 던졌다. 그로인해 잘라첸코의 외관은 심각하게 훼손되었고 만성질환을 앓게되었다. 후에 잘라첸코가 리스벳을 땅에 묻어버리기 전 지금까지 거울을 볼때마다 리스벳을 생각하며 증오한다고 했다.\n리스벳의 방화이후, 법적으로 정신이상자로 판결나고 웁살라에 위치한 소아정신병원에 수감입원되었다. 그곳에서 리스벳은 심리학자 닥터 피터 텔레보리안(Dr. Peter Teleborian)의 감시아래에 지내게 되었는데, 그 또한 잘라첸코와 관련이 있었으며 잘라첸코의 신분을 덮어두기 위해 모든것을 알고있는 리스벳을 평생 정신병원과 법원의 통제안에 가두어두려했다.\n리스벳이 웁살라의 정신병원에 입원했을 당시에는 텔레보리안의 감독아래 억제요법을 받았는데 이것은 아동성장애자인 텔레보리안의 성적욕구를 리스벳을 통해 풀기위함이었다. 텔레보리안은 리스벳을 법적으로 정신병자로 구분하였는데, 이는 자신들이 한 행동들을 묻어주기 위함이었다. 그리고 팔그렌의 뇌졸중 이후 자신들과 한패인 비여만을 리스벳의 후견인으로 임명함으로, 리스벳을 자신들의 감독안에 두기위해서였다."
1956,"스웨덴어는 인도유럽어족 게르만어파의 한 갈래인 북게르만어군에 속한다. 덴마크어와 함께 동스칸디나비아어 그룹에 속하며, 페로어, 아이슬란드어, 노르웨이어가 속하는 서스칸디나비아어 그룹과는 구분된다. 최근 연구에서는 노르웨이어를 동스칸디나비아어 그룹에 포함시켜 북게르만어군을 섬 스칸디나비아어와 본토 스칸디나비아어로 분류하는 경향도

In [36]:
special_char = "Þ"
train_df[train_df["context"].str.contains(special_char)]["context"]

,context
517,"노이는 이름 모를 아이슬란드 서쪽 외딴 어촌 마을에서 그의 할머니 리나 (Anna Friðriksdóttir)와 함께 사는 17살 소년이다. 아버지 키디 (Þröstur Leó Gunnarsson)는 알코올 중독자의 택시 운전수로 같은 마을에 살지만, 노이와 가까운 관계를 유지하진 못한다. 알비노인 그의 외모는 마을에 다른 사람들과 완전히 다르다. 그는 대부분의 시간을 황량한 마을이나 동네 서점을 어슬렁거리거나, 그만의 사적인 공간인 할머니 집의 비밀 공간에서 보낸다. 산으로 둘러 쌓여있고, 겨울에는 눈에 갇혀 산을 통과하는 도로가 막혀 오직 배만이 외부와의 접촉을 허락되는 마을에서 노이는 고통스러워한다. 지적으로 뛰어나지만, 그는 학업에는 완전 무관심하며 선생님과의 관계도 좋지 않다. 결석을 밥 먹듯이 하는 그는 주로 동네 주유소에 있는 슬롯머신을 조작하여 돈을 챙긴다. 노이에게 이 절망적인 마을에서의 미래는 불안할 뿐이다.\n\n노이의 상황은 마을에 새로 온 매력적인 주유소 점원 아이리스 (Elín Hansdóttir)를 만나며 변한다. 노이는 아이리스와의 서툰 연애를 시작한다. 어느날 밤 그들은 지역 자연사 박물관에 몰래 들어간다. 경비원에게 쫓겨 창고에 숨게 되었는데, 그 곳에서 그들은 불 빛이 나는 세계지도를 발견한다. 함께 지도를 보다 아이리스는 이 곳에서 벗어나자고 제안한다. 그 후 노이는 아이리스와 함께 마을과 아이슬란드를 떠나는 꿈을 갖게 된다. 한편, 생일을 맞아 할머니에게로부터 그는 열대 섬 사진이 들어있는 뷰 마스터 View-master를 받는다. 그는 자신 주변의 환경과 정반대인 열대 해안의 이미지에 얼어붙는다.\n\n그러던 어느 날, 수학 시간에 자신 대신 녹음기를 출석 시킨 노이는 선생을 화나게 만들었다. 선생은 교장에게 그를 퇴학시킬 것을 부탁하고, 노이는 결국 학교에서 쫓겨나게 된다. 사실을 알게 된 아버지는 처음엔 그에게 분노했지만, 미안해하며 곧 그를 마을 술집에 데려갔다. 하지만 술집에서 노이는 쫓겨나게 되고, 그날 밤 그는 아이리스를 만나러 그녀의 집에 갔지만 그녀의 아버지 오스카에게 발각된다. 오스카의 충고에도 불구하고, 아이리스는 노이가 밤새 그녀의 집에 머물 수 있도록 도와준다.\n\n한편, 그의 할머니는 마을 점쟁이 길피 (Kjartan Bjargmundsson)에게 노이의 운세를 봐주고, 그가 바른 길로 갈 수 있도록 도와달라고 부탁한다. 길피는 노이에게 죽음의 운명이 보인다고 말한다. 노이는 길피가 자신을 놀린다는 생각에 화를 내며 떠난다. 그 길로, 노이는 일을 관두고 할머니의 총을 꺼내 달아난다. 그는 은행을 털고자 했지만, 아무도 그를 심각하게 여기지 않았고 총을 빼앗으며 내쫓았다. 그는 다시 은행으로 돌아와 그의 계좌에서 돈을 꺼내 새로운 양복을 구입한다. 그는 새로운 옷을 입고 차를 훔쳐 아이리스에게 떠나자고 제안했다. 그러나 아이리스는 혼란스러워 하며 그와 떠나기를 망설였다. 결국 노이는 혼자 떠나게 되었지만, 곧 차는 눈에 갇혀 경찰에게 붙잡힌다.\n\n경찰서에서 돌아온 노이는 그만의 비밀 공간으로 내려간다. 그 순간 갑자기 땅이 흔들리며 빛이사라진다. 노이는 탈출하려 했지만 머리 위에 있는 문은 열리지 않고 꼼짝없이 갇히게 되었다. 결국 그는 외부의 도움으로 빠져 나오게 되었고, 그것이 산사태이었음을 알게 된다. 산사태로 그의 집은 무너졌고 아버지와 할머니의 목숨을 앗아갔다. 구조 쉼터에 있던 노이는 산사태로 아이리스와 길피를 포함한 그가 알고 있던 모든 사람이 죽었다는 것을 알게 된다. 시간이 지나고 그는 할머니 집터에서 그의 뷰 마스터를 발견하여 그것을 들여다 보았다. 영화는 노이가 바라보고 있던 열대 해변 사진이 천천히 진짜 해변의 모습으로 바뀌면서 끝난다."
1147,"토르그뉘르 로그마드(Þorgnýr lǫgmaðr)는 중세 초기에 살았던 로그마드들로, 토르그뉘르라는 이름을 가진 로그마드는 세 명 있다. 그 출전은 스노리 스투를루손의 헤임스크링글라, 그리고 그보다는 덜 알려져 있는 스비아인의 대전사 스튀르뵤른의 사트르, 바보 흐로이의 사트르가 있다. 세 명의 토르그뉘르는 모두 티운달란드의 로그마드였으며 스웨덴 왕국을 섬겼다. 이 이야기들이 모두 역사적 사실이라면, 세 명의 토르그뉘르 로그마드는 3대 부자 관계다.\n\n가장 유명한 토르그뉘르는 헤임스크링글라에 그 행적이 전하는 인물로, 스웨덴의 올라프 쇤스키와 노르웨이의 올라프 헬가의 치세에 살았다. 스노리에 따르면, 티운달란드에 토르그뉘르라는 이름의 로그마드가 살았는데, 그 토르그뉘르의 아버지는 토르그뉘르의 아들 토르그뉘르(Þorgnýr Þorgnýrson)이었다. 토르그뉘르의 가계는 오랫동안 티운달란드의 로그마드로서 많은 왕들을 섬겼다. 토르그뉘르는 늙었을 때 스웨덴에서 가장 지혜로운 사람으로 여겨졌으며, 라근발드 울프손의 양부였다.\n\n당시 스웨덴과 노르웨이의 두 명의 올라프 왕이 서로 전쟁을 벌였다. 1018년, 뵤른 스탈라레와 햘티 스케기아손 등 노르웨이 대표단이 웁살라 팅그에 와서 스웨덴 왕녀 잉게게르드 올로프스도테르를 노르웨이 왕비로 내놓으라고 요구했다. 라근발드 울프손도 이를 거들었다. 올로프 쇤스키는 노하여 라근발드를 추방하려 했다. 그러나 라근발드의 양부 토르그뉘르가 스웨덴에서 가장 존경받는 원로로서 라근발드의 역성을 들었다. 토르그뉘르가 조리있게 일장연설을 하자 그 자리의 전원이 박수를 치며 환호했다. 결국 올로프 쇤스키는 딸을 내주겠다고 하며 노르웨이와 평화를 맺었으나, 나중에 그 약속을 어겼다.\n\n올라프 뵤른손의 아들이며 에이리크 인 시그르셀리의 조카인 스튀르뵤른 스테르키에 관한 단편인 스비아인의 대전사 스튀르뵤른의 사트르에는 다른 토르그뉘르 로그마드가 나온다. 이 토르그뉘르 로그마드는 헤임스크링글라의 토르그뉘르 로그마드의 아버지인 것 같다. 이 토르그뉘르는 매우 늙어서 눈도 거의 멀었고, 에이리크와 스튀르뵤른 사이에 벌어진 전쟁에 참여하지 못했다. 하지만 소떼와 말떼가 데인인 군대 쪽을 향해 달려가도록 하여 싸움에 도움을 주었다. 이 가축 떼는 스튀르뵤른 측에 큰 피해를 입혔다."
1547,"《암탉 토리르의 사가》(Hænsa-Þóris saga|헨사토리스 사가)는 아이슬란드 사가 작품 중 하나이다. \n\n주인공 헨사토리르(Hænsa Þóris)는 장사로 부유해져 땅을 사게 된 사람으로 주위에 인망이 없었다. 이야기 내내 토리르는 보다 제대로 된 배경을 가지고 있는 이웃사람들과 부정적으로 비교된다. 이웃들이 겨울나기에 필요한 짚단을 팔라고 요구하자 토리르는 이를 거부한다. 그러거나 말거나 이웃을이 멋대로 짚단을 가져가 버리자 토리르는 이웃들의 집에 불을 질러 그들을 산채로 태워 죽인다. 이에 대한 복수가 뒤따라 토리르도 살해당해 목이 잘리는 것으로 이야기가 끝난다.\n\n《암탉 토리르의 사가》는 중세 아이슬란드 문화의 여러 면을 보여준다. 예컨대 나그네를 환대해야 하고 이웃들에게 관대해야 하며 사법적 판단을 위해서는 추장의 지지를 얻어야 하는 것 등이 그러하다. 또다른 학설로는 당시 노르웨이 국왕 마그누스 6세가 법률을 개정한 것과 이 작품이 유관하다고 한다. 새로운 법에 따르면 농부는 이웃들이 절박한 필요로 한다면 그들에게 건초를 팔아야만 하는 의무가 있었다. 건초 판매를 거부하면 벌금을 물었고, 무력으로 저항한다면 이웃들이 그를 공격해도 범죄로 취급되지 않았다. 이는 아이슬란드 전통 법제사에 유례가 없던 새로운 법으로서, 별로 인기가 없었던 것으로 보인다. 이 학설에 따르면 《암탉 토리르의 사가》는 새 법률을 홍보하기 위한 선전물로서 쓰여진 것이다."
3496,"토라 보르가르효르트(Þóra Borgarhjǫrtr)는 노르드 신화에 나오는 인물이다. 라그나르 로드브로크의 두 번째 아내이며 예탈란드 야를 헤라우드의 딸이다.\n\n토라는 베스테르예탈란드의 내실에 살았다. 아버지 헤라우드는 토라에게 작은 린트부름을 애완동물로 주었는데 린트부름이 너무 커져서 내실을 통째로 휘감을 지경이 되었고 토라는 그 안에 갇혀 버렸다. 헤라우드는 누구든 린트부름을 퇴치하는 남자와 토라를 결혼시키겠다고 약속했다.\n\n첫 번째 부인 라드게르타와 이혼한 뒤였던 라그나르는 

In [37]:
special_char = "क"
train_df[train_df["context"].str.contains(special_char)]["context"]

,context
217,"노예제도가 폐지 된 후, 유럽 국가의 많은 식민지와 미국에서 노동력이 부족했다. 영국의 식민지였던 인도 대륙의 빈민층과 아편전쟁 후, 광둥성, 푸젠성 두 성을 중심으로 산터우, 샤먼, 마카오 등지에서 저임금의 노동력을 가진 쿨리가 세계 각지에 보내졌다. 처음에는 ‘인도 노동자’(क़ुली)를 가리키는 호칭이었지만 나중에 중국인 노동자들에게 “쿨리”(苦力)라는 한자가 음역되었다. 미국은 대륙횡단철도 건설 노동자 등으로 고용되었고, 중국에서 캘리포니아로 10만명 이상이 보내졌다. 호주와 말레이시아, 마다가스카르 등에도 각각 10만명 정도가 이주했다고 한다. 공식적인 해외 유입은 베이징 조약 체결 이후가 되었지만, 그 이전에도 사실상 중국에서 쿨리 수출은 이루어지고 있었다. 그 배경에는 중국의 인구 증가, 태평천국 이후의 사회적, 시대적 불안이 있었다. 쿨리는 열악한 환경에서 생활했기 때문에 항해 중 또는 작업 중에 사망하는 경우가 많았고, 현지에서도 최하층의 생활을 강요당했지만, 자손들 중에는 그러한 환경을 벗어나 사회 신분이 상승된 이도 적지 않았다. 또한 1937년 중일전쟁 때에는 만주와 일본의 점령지에서도 쿨리가 사역을 했지만, 중국이 공산화되면서 사실상 쇄국 정책이 채택되고, 쿨리 무역은 종결되었다."


In [38]:
special_char = "ㄱ"
train_df[train_df["context"].str.contains(special_char)]["context"]

,context
661,"정무공 오정방(1588∼1634), 천파공 오상, 충정공 오두인(1624∼1689) 등 해주 오씨의 이름난 학자들을 배출한 유서깊은 곳이다. 특히 충정공은 이 집에서 성장하였으며 나라에 큰 공헌을 한 일이 있어 우암 송시열이 글을 써서 보내오기도 하였다. 중종 5년(1510) 덕봉리에 처음 세웠으나 효종 1년(1650)에 지금의 위치로 옮겨 지었다. 현재 문간채, 안채 겸 사랑채, 사당이 남아있다.\n\n이 집은 일반적으로 안채와 사랑채가 떨어져 있는 것과는 달리 연속하여 하나의 건물에 구성하였는데, 특히 ㄱ자형 몸채에는 일자형으로 부분을 길게 연장하고 중간에 담을 두어 안팎으로 나눈 수법은 이 집의 특징이다.\n\n안채 부분은 대청과 건넌방, 안방, 부엌 등을 두었고 사랑 부분은 툇마루를 두고 기단을 높게 하여 개방성을 두드러지게 나타냄으로써 실제 규모에 비해 장중하게 느껴진다. 사당은 사랑채 뒤쪽의 한쪽에 별도로 담장을 둘러 일곽을 이룬다. \n\n가옥의 배치 및 구성과 함께 건물의 구석구석에서 느낄 수 있는 목수의 안목과 솜씨가 높이 평가되는 주택이다.\n\n이 건물은 공간의 다양성을 살리면서 남녀의 생활 공간을 적절히 구분하였고, 전체적으로 소박하게 꾸민 편이다. 조선 후기 사대부 가옥의 구조를 잘 보여주고 있으나 안채와 사랑채를 엄격히 구분하였던 17세기 사대부 가옥의 일반적인 경향과는 달리 안채와 사랑채를 연결한 독특한 구조를 보인다. 조선 후기 주택사 연구에서 중요한 자료로 꼽힌다."
2247,"안채와 사랑채, 행랑채와 곳간채, 문간채로 구성되어 있으며 1938년에 지은 집이다.\n\n안채는 앞면 5칸·옆면 3칸이며, 지붕은 옆면에서 볼 때 여덟 팔(八)자 모양을 한 팔작지붕이다. 천장 위에는 모두 다락을 두었고 앞면과 뒷면 창은 빗살창으로 꾸몄다.\n\n사랑채는 안채 앞쪽에 배치되어 있는 건물로 지붕은 팔작지붕이다. ‘영춘헌’이라는 현판이 걸려 있으며, 안채와 같이 천장 위에 다락을 만들었고 ㄱ자형으로 뻗친 대청누각이 있다.\n\n곳간채는 안채와 사랑채 사이에 남북으로 뻗어 있다. 앞면 4칸·옆면 1칸 규모로 지붕은 옆면에서 볼 때 사람 인(人)자 모양을 한 맞배지붕이다.\n\n행랑채는 곳간채 낮은 편에 서 있는데 앞면 4칸·옆면 1칸 크기에 지붕은 맞배지붕으로 꾸몄다.\n\n안채와 사랑채를 통하는 안문은 팔작지붕이며 그 옆에 조그마한 문이 있다. 문의 윗부분을 자연스럽게 타원형으로 짠 것이 인상적이다.\n\n문간채는 앞면 3칸·옆면 1칸으로 대문을 중심으로 좌우에 하나씩 방이 있으며 대청 뒤쪽에서 남으로 뻗은 누각 형태의 마루가 시원한 느낌을 주고 있다."
3062,"나지막한 언덕에 자리하고 있는 옛집으로, 조선 숙종(재위 1674∼1720) 때 이조판서를 지낸 이자가 고향으로 돌아와 지은 것이라 한다. 지은 연대는 정확히 알지 못하며, 90여 년 전에 안채를 크게 수리하였다고 한다.\n\n안채와 사랑채가 ㅁ자형으로 배치되었고, 동남향을 하고 있다. 1m 정도의 비교적 높은 기단 위에 지었으며, 여덟 팔(八)자 모양의 화려한 팔작지붕집이다. 안채는 영동지방에서 흔히 볼 수 있는 겹집형에, 영서·중부지방에서 일반적인 대청과 건넌방이 ㄱ자형으로 접하여 있다. 대청에는 모두 문을 달았고, 건넌방과 대청·안방부분에도 툇마루를 달아 외부와 연결을 주었다.\n\n부엌의 앞면에서 광·외양간과 대문·사랑채가 ㄷ자형으로 이어졌다. 사랑채는 큰사랑·작은사랑·서함·대청·머슴방으로 구성되었고, 특히 외양간이 대문 오른쪽에서 돌출된 것이 특이하다. 사랑채의 대청에도 문을 달았고, 앞뒷면에 툇마루를 두어 시원한 공간구성을 하고 있다.\n\n사랑채의 서쪽에는 별채인 수고당이 있으며, 대청·작은마루·다락·서고로 구성되어 있다. 수고당에는 외재 이단하 내외분의 옷(국가민속문화재 제4호)이 있어 조선시대 의복연구에 중요한 자료가 되고 있으며, 이 외에 각종 시문초고·교지 및 전적과 간찰·서화가 보존되어 있다."
3372,"2020년 3월 27일 전북 전주시에 사는 40대 ㄱ씨가 군산의 한 병원에 진료차 방문했고 그 병원에선 ㄱ씨의 해외여행 이력을 들어 코로나 검사를 받아야 진료를 받을 수 있다 안내하며 군산보건소 선별진료소로 보냈고 ㄱ씨가 보건소 직원에게 ""전주에 사는데 군산에서 검사 받아도 되느냐?"" 묻자 직원이 괜찮다고 답해 1시간 가량 기다렸으나 직원이 자기 주소지에서 검사받아야 비용면제하도록 방침 바뀌었다며 전주에서 검사받기 권하자 ㄱ씨가 ""왜 미리 안내해 주지 않았느냐"" 며 언성 높이고 화를 내었고 주변의 직원들이 ""여기 시장님이 계시니 조용히 해달라""고 시장을 의식하는 말만 했다하며 ㄱ씨는 자신의 승용차에 타서 집에 돌아가버리려는데 보건소 한 직원이 ㄱ씨 차를 막아서며 오해 풀자고 이야기하던 와중 당시 보건소에 있던 강 시장이 떠난다는 소리를 듣자 그 직원이 ㄱ씨와 이야기를 끊고 시장 차량으로 향해버리자 ㄱ씨가 다시 언성을 높이며 ""시장이 간다고 사람을 세워두느냐? 난 시장 낮짝도 모른다. 시장은 사람이고 시민은 사람이 아니냐?"" 고 이야기하자 강 시장이 차에서 내리며 ""내가 시장이다 XX야. 어린놈의 XX. 뚫린 입이라고 싸가지 없게. 저런 것은 집어넣어 버려야 해"" 이런 식으로 욕을 했다는 것이다. 그리고 주변 직원들이 ㄱ씨를 움직이지 못하게 막고 그 사이에 시장이 차를 타고 빠져나갔다는 것인데 이에 대해 ㄱ씨가 한 SNS에 ""시민을 생각하고 시민의 소리를 듣고 시정활동 해야하는 사람이 시민에게 욕설과 폭언 하는 게 말이 되느냐? 지나가던 시민들도 그 상황을 봤을 것이다. 저 어린 놈 아니다. 고등학생 자녀가 있고 마흔이 넘은 나이다"" 라고 글을 올렸고 그걸 본 사람들이 댓글로 시장을 비난하자 군산시 직원들이 ㄱ씨에게 연락해 사과했고 나중에 강 시장의 사과전화를 받고서야 마음을 풀었다 한다.\n\n강 시장은 언론들과의 통화에서 코로나 사태 때문에 보건소 직원들이 고생하는데 ㄱ씨가 고함지르는 거 보고 감정 추스르지 못하고 실수를 범했다며 ㄱ씨를 만나 오해를 풀었다며 이런일이 일어나 송구하다고 밝혔다. 조선일보 한겨레"


In [39]:
special_char = "ㅎ"
train_df[train_df["context"].str.contains(special_char)]["context"]

,context
367,"2010년 4월 30일, 라리사는 29세 센터백과 협상에 성공하여 이 해 여름에 녹색 군단과의 계약이 만료되면 자유이적으로 팀에 합류하게 될 것이라 발표하였다. 프랑스에서 황소같은 플레이스타일로 '황소' (Le Taureau) 라는 별칭을 얻은 그는 릴과 생테티엔 선수로 활약해 리그 1에서 성공한 그리스 선수이나, 알랭 페랭 감독에 의해 스타드 조프리 귀샤르 연고 팀의 명단에서 밀려났다. 전 올랭피크 리옹 감독이 경질된 후, 타블라리디스는 크리스토프 갈티에르 감독 하에 인상을 줄 기회를 잡았으나, 갈티에르 임기의 첫 경기인 마르세유전에서 10분만에 퇴장당하면서 녹색 군단으로써의 ㅎ뢀약이 사실상 끝나버렸다. 나중에 합류할 구단의 공식 웹사이트에서, 그리스 인터넷 언론에 그는 ""저는 제 경력을 해외에서 이어나가는 것을 우선순위로 잡았었습니다. 저는 그리스에 복귀할 여부를 4월까지 결정했으며, 9년만의 복귀는 제가 가족을 거느리는 것으로 인해 만반의 준비를 필요로 합니다. 저는 그리스의 서너개 구단의 추천을 받았으나, 심사숙고 끝에 저는 라리사에 입단하기로 결정했습니다."" 타블라리디스는 전에 1월에 그리스 클럽의 진심한 제의를 받았으나, 차남의 출생을 지켜보기 위해 거절했었다. 그는 지난 시즌을 8위로 마친 수페르리가 엘라다 클럽과 4년 계약을 맺었다.\n\n그러나, 2011년 8월 9일, 소속팀이 풋볼 리그로 강등당함에 따라 계약을 해지시켰다."
3684,"모아주기는 한글 글자를 초성, 중성, 종성 순서에 상관없이 오토마타가 지능적으로 합성해 주는 기능이다. 모아치기는 문자적으로 키보드 입력을 짧은 시간의 차이 안에 동시에 입력하는 경우를 뜻하며, '동시'에 입력되는 키를 같은 음절로 결합해주는 기능이다. 모아치기의 반대는 이어치기이다. 모아치기의 이론은 있었지만, 실제 발명은 안마태가 최초라고 주장된다. 그러나 속기 자판에서는 이미 초/중/종성의 모아치기를 하고 있었으므로 이 주장은 논란의 여지가 있으며, 일반 키보드에 속기 자판의 장점을 활용하였다고 보아야 한다. 모아치기와 이에 대한 세부적인 용어는 아직 공식적으로 확실하게 정해지지 않았다. \n\n# 이어치기(순차입력): 글자를 음소 단위로 하나씩 쳐서 입력하는 방식을 말한다. (예: 한 = ㅎ + ㅏ + ㄴ받침) 풀어치기라고도 하는데, 그렇게 말하면 풀어쓰기와 혼동할 여지가 있다. 그래서 날개셋 입력기의 경우는 '이어치기'라는 용어를 사용한다.\n# 모아치기: 함께 입력된 음절의 모든 자모 글쇠를 같은 음절로 모아주는 기능이다. 한 음절을 구성하는 모든 글쇠를 한 타로 동시에 쳐서 입력하는 것을 말한다. 안마태 연구소에서는 이에 대해 '동시치기'라고 정의했다 (예: 한 = ㅎ/ㅏ/ㄴ받침 한번에) 안마태 소리글판이나 속기 자판 등의 모아치기용 자판만 가능.\n# 부분 모아치기(나눠치기): 함께 입력된 음절의 일부 자모 글쇠를 같은 음절로 모아주는 기능이다. 즉, 한 음절을 두 번 이상으로 나누어서 입력한다. 안마태 연구소에서는 한 음절을 두 번에 나누어 넣는 것에 대해서만 '모아치기'라고 정의했다. (예: 한 = ㅎ/ㅏ + ㄴ받침) 세벌식 자판만 가능.\n# 모아주기: 시차와 관계 없이 입력된 글쇠를 같은 음절로 모아주는 기능이다. 손으로 모아서 친다는 의미가 아닌 오토마타가 글자를 모아준다는 의미로 많이 쓴다. 따라서 이 경우, 모아치기는 잘못된 표현임이 분명하다. 모아주기가 뜻에 맞을 것이다."


In [40]:
special_char = "ㄹ"
train_df[train_df["context"].str.contains(special_char)]["context"]

,context
2656,"다른 언어로 노래하기 위한 목적으로 노랫말을 번역하는 것을 노래 번역으로 부르기도 한다. 노래 번역은 음성이 담긴 음악이 대부분으로 구성된 노래를 다루기에 시 번역과 밀접한 연관성을 가지며 압운과 각운 등 여러가지 운율도 고려되어야 한다. 19세기 이래로 산문과 자유 시 구조가 일부 예술적 음악 장르에서 다루어 졌지만 대중 음악에서는 운율(스탠자)을 유지함에 있어서는 상당히 보수적인 입장을 취했다. 시를 노래를 위해 번역한 가장 기초적인 예가 찬송가이다. 독일의 합창성가를 영어로 옮긴 주번역자가 캐서린 윙크워스이다. \n\n노랫말의 번역은 대개 시구 번역보다 훨씬 제약이 많다. 이는 형태에 있어 거의 혹은 전혀 자유가 없어 다양한 번역을 창조해내기가 불가능하고 시구 구조에서도 다른 여지를 찾기 어려운 탓이다. 누군가는 각운을 번역 과정에서 생략하거나 수정해낼 수 있겠지만 각 음절의 번역이 특정한 음표를 나타내는 음악적 기조를 따라가려면 번역가는 엄청난 노력을 들여야 한다. 산문처럼 길게 늘어뜨린 노랫말의 경우 시구처럼 짧은 노랫말보다는 어려움이 덜할 수 있는 이유가 음운을 맞추기 위해 음절을 이곳 저곳에서 떼거나 분할해 적용할 수 있기 때문이다. 그런 사실을 인정하더라도 과정상 엄밀히 말해 시구 번역에 해당하는 노랫말 번역은 가능한한 노래의 음절을 맞춰 진행해야하는 노래의 특성상 제약이 많을 수 밖에 없다.\n\n다른 고려사항으로는 노랫말 번역시 단어와 글귀를 반복하거나 나머지를 대체하는 행위 혹은 구두점, 자음의 사용에 있어 고음에 맞춰 사용하기 등이다. 이외에도 리듬에 맞춰서 사용했을 시 도착어에서 부를 때 훨씬 부드럽고 자연스럽게 적용될 수 있을 자음을 택하는 것도 이러한 작업에 해당한다. 노랫말 번역은 상당히 그리고 완벽하게 원래판과 달라질 수 있다. 예를 들어 영어로 된 노래를 한국어로 번역할 때 ㄹ이나 ㄴ 처럼 콧소리를 유발하는 음절이 계속해서 나오거나 발음이 꼬일 경우에는 다른 유사한 의미의 단어를 취하거나 대체해 노랫말이 간결하게 전달될 수 있도록 돕는 일도 포함될 수 있다.\n\n노랫말의 번역은 노래를 하거나 좀 더 혹은 덜 문학적 형태를 띨 수 있겠지만 청중이나 가수, 지휘자를 위한 배려의 목적에서 고려되기도 한다. 잘 모르는 언어로 설명될 때가 그렇다. 가장 흔히는 번역이 자막이나 오페라의 대화가 번역되어 영사기로 보이는 것에 해당한다. 콘서트 프로그램 진행에도 적용되며 상업 오디오 CD-ROM에도 이런 경우가 발견된다. 더욱이 전문적이거나 아마추어 가수들이 보통 잘 모르는 언어로 노래할 때 의미를 이해할 수 있도록 번역하여 부르거나 번역을 이용하는 모습을 볼 수 있다."
3690,"다른 언어로 노래하기 위한 목적으로 노랫말을 번역하는 것을 노래 번역으로 부르기도 한다. 노래 번역은 음성이 담긴 음악이 대부분으로 구성된 노래를 다루기에 시 번역과 밀접한 연관성을 가지며 압운과 각운 등 여러가지 운율도 고려되어야 한다. 19세기 이래로 산문과 자유 시 구조가 일부 예술적 음악 장르에서 다루어 졌지만 대중 음악에서는 운율(스탠자)을 유지함에 있어서는 상당히 보수적인 입장을 취했다. 시를 노래를 위해 번역한 가장 기초적인 예가 찬송가이다. 독일의 합창성가를 영어로 옮긴 주번역자가 캐서린 윙크워스이다. \n\n노랫말의 번역은 대개 시구 번역보다 훨씬 제약이 많다. 이는 형태에 있어 거의 혹은 전혀 자유가 없어 다양한 번역을 창조해내기가 불가능하고 시구 구조에서도 다른 여지를 찾기 어려운 탓이다. 누군가는 각운을 번역 과정에서 생략하거나 수정해낼 수 있겠지만 각 음절의 번역이 특정한 음표를 나타내는 음악적 기조를 따라가려면 번역가는 엄청난 노력을 들여야 한다. 산문처럼 길게 늘어뜨린 노랫말의 경우 시구처럼 짧은 노랫말보다는 어려움이 덜할 수 있는 이유가 음운을 맞추기 위해 음절을 이곳 저곳에서 떼거나 분할해 적용할 수 있기 때문이다. 그런 사실을 인정하더라도 과정상 엄밀히 말해 시구 번역에 해당하는 노랫말 번역은 가능한한 노래의 음절을 맞춰 진행해야하는 노래의 특성상 제약이 많을 수 밖에 없다.\n\n다른 고려사항으로는 노랫말 번역시 단어와 글귀를 반복하거나 나머지를 대체하는 행위 혹은 구두점, 자음의 사용에 있어 고음에 맞춰 사용하기 등이다. 이외에도 리듬에 맞춰서 사용했을 시 도착어에서 부를 때 훨씬 부드럽고 자연스럽게 적용될 수 있을 자음을 택하는 것도 이러한 작업에 해당한다. 노랫말 번역은 상당히 그리고 완벽하게 원래판과 달라질 수 있다. 예를 들어 영어로 된 노래를 한국어로 번역할 때 ㄹ이나 ㄴ 처럼 콧소리를 유발하는 음절이 계속해서 나오거나 발음이 꼬일 경우에는 다른 유사한 의미의 단어를 취하거나 대체해 노랫말이 간결하게 전달될 수 있도록 돕는 일도 포함될 수 있다.\n\n노랫말의 번역은 노래를 하거나 좀 더 혹은 덜 문학적 형태를 띨 수 있겠지만 청중이나 가수, 지휘자를 위한 배려의 목적에서 고려되기도 한다. 잘 모르는 언어로 설명될 때가 그렇다. 가장 흔히는 번역이 자막이나 오페라의 대화가 번역되어 영사기로 보이는 것에 해당한다. 콘서트 프로그램 진행에도 적용되며 상업 오디오 CD-ROM에도 이런 경우가 발견된다. 더욱이 전문적이거나 아마추어 가수들이 보통 잘 모르는 언어로 노래할 때 의미를 이해할 수 있도록 번역하여 부르거나 번역을 이용하는 모습을 볼 수 있다."


# question과 answer에도 다른 언어가 존재할까?

In [54]:
answers = train_df["answer_text"]
print(len(answers))
answers.head()

3952


,answer_text
0,하원
1,《경영의 실제》
2,백성
3,중국
4,4개


In [67]:
# 특수 문자를 찾는 함수 정의
def find_special_characters(text, pattern):
    # 정규 표현식을 사용하여 특수 문자 찾기
    return re.findall(pattern, text)

# context에서 모든 특수 문자를 찾고 중복 제거
all_special_chars = set()
pattern = r'[^a-zA-Z0-9가-힣\s]' # 알파벳, 숫자, 한글, 공백 제외
#pattern = r'[^a-zA-Z0-9가-힣\s\u4e00-\u9fff]' # 알파벳, 숫자, 한글, 한자 공백 제외
#pattern = r'[^a-zA-Z0-9가-힣\s\u4e00-\u9FFF\u3040-\u309F\u30A0-\u30FF\u0600-\u06FF]' # 알파벳, 숫자, 한글, 한자, 일본어, 아랍어 공백 제외
for idx, context in enumerate(train_df['answer_text']):
    special_chars = find_special_characters(context, pattern)
    all_special_chars.update(special_chars)

# 결과 출력
print(f"찾은 특수 문자 종류: {all_special_chars}")
print(f"특수 문자의 개수: {len(all_special_chars)}")

찾은 특수 문자 종류: {'月', '之', '流', '宋', '片', 'な', '陽', '祚', '意', '忠', '資', '勳', '駅', "'", '魯', 'に', '蔣', '卵', '務', '」', '愛', '弼', '°', 'よ', '\xad', '>', '堂', '成', '一', '呂', '祠', '興', '儼', '＞', '趙', '部', '〈', '寺', '北', '城', '京', '扶', '蓮', '“', '重', '王', '式', '槪', '鉱', '+', '淵', '＜', '超', '有', '『', '河', '信', '鄭', '淸', '念', '餘', 'ㅋ', 'の', 'し', '牲', '海', '次', '/', '≫', '匡', '二', '藏', '張', ',', '康', '琿', 'ら', '汪', '‘', ')', '〉', '”', '毅', '學', '劉', '》', '心', '.', '原', '總', '雲', '限', '狼', '循', '性', '監', '"', '田', '理', '·', '身', '割', 'ä', '害', '祭', '大', '輪', '軍', '分', '麻', '塡', '甚', '陳', '華', '!', '臺', '奎', '縣', '《', '辣', '坂', '釋', '日', '郵', '∧', ':', '≪', '格', '金', '壇', '受', '夔', '塔', '遷', 'う', '<', '優', '’', '・', '太', '(', '陵', '』', '池', '「', '刀', '-', '媒', '前', '識', '肥', '冬', '公', 'い', '南', '~', '%', '孟', '文', '平'}
특수 문자의 개수: 160


In [70]:
# \xad가 어떤식으로 포함되어 있는지 확인
train_df[train_df["answer_text"].str.contains("\xad")]

,title,context,question,answer_start,answer_text,document_id,languages
544,강중인,"경상북도 출신으로 1935년에 경성제국대학 법문학부 법학과를 졸업했다. 1938년에 경성지방법원 및 경성지방법원검사국 사법관시보가 된 것을 시작으로 법조계에서 근무를 시작했다.\n\n1939년에는 경성지방법원 검사대리가 되었고, 1940년에 예비검사를 거쳐 그해부터 1943년까지 조선총독부 검사로 근무했다. 강중인은 경제담당 검사로 일하면서 태평양 전쟁 중 시국 연설과 기고로 사회 활동을 병행했다. \n\n1942년에 조선임전보국단 주최로 경성중앙방송국에서 저명 인사의 시국연설을 방송했을 때, 〈필승사상전(必勝思想戰)〉이라는 제목으로 연설한 일이 있다.\n\n《삼천리》에 〈최근의 반도의 경제사범 ­국민의 신경제 윤리의 파악을 위하여­〉라는 글을 기고하기도 했다. 이 글에서는 경제사범을 ""미증유의 국난""을 돌파해 ""대동아를 건설하는"" 국가적으로 중대한 순간에 사리사욕에만 눈이 먼 집단으로 묘사하고, 이들은 국민의 일대수치이므로 엄벌주의로 대처할 것이라 경고했다. 또한 관민이 일체가 되어 ""일사보국의 성을 다함""으로써 성업을 어서 완수해야 한다는 주장을 폈다.\n\n1943년에 대전지방법원 검사로 자리를 옮겨 일본 제국이 태평양 전쟁에서 패망할 때까지 근무했다. \n\n미군정 하의 서울에서 변호사를 개업하면서 좌익 운동에 뛰어들었다. 좌익 법조인 모임인 법학자동맹과 남조선로동당에 가입해 활동했다. 강중인이 남로당에 가입하게 된 동기는 후에 법정에서 다음과 같이 밝힌 바 있다. \n\n일정(日政) 당시에는 내가 내 한 몸을 구하기에 여력이 없었습니다. 8·15 해방을 맞이하자 피고는 이때야 깨달은 바가 있었습니다. 이는 다름이 아닙니다. 내가 어찌하면 국가와 민족을 위하여 일할 수 있을까 함이었습니다. 해외에서 국가와 민족을 위하여 투쟁하던 위대한 애국자들이 해방된 조국을 찾아 들어옴을 볼 때 무어라고 표현할 수 없는 느낌이 있었습니다. 그 후 피고는 남로당에 가입을 작정하였던 것이며, 법맹에도 초창기부터 가입하였습니다.\n\n1946년 민주주의민족전선 토지문제연구위원에 임명되었고, 미군정 과도정부의 사법부 총무국장도 지냈다. 같은 해 7월에 정판사 위조지폐 사건이 발생하여 남로당과 미군정이 정면 충돌을 일으켰을 때 위폐 사건 피고인들의 변호를 맡았다. 남로당은 곧 불법화되었고, 강중인은 1949년에 발생한 법조프락치 사건으로 구속되었다. 1심 재판에서 징역 3년형을 언도받고 항소 중에 한국 전쟁이 발발하여 조선민주주의인민공화국으로 갔다.\n\n2008년 민족문제연구소가 발표한 민족문제연구소의 친일인명사전 수록예정자 명단 중 사법 부문에 포함되었다.",강중인이 경제사범을 비판하며 국민의 일대수치라 표현한 글은?,324,최근의 반도의 경제사범 ­국민의 신경제 윤리의 파악을 위하여­,15575,"Korean, English, Chinese"


In [71]:
# 해당 언어거 어떤식으로 포함되어 있는지 확인
train_df[train_df["answer_text"].str.contains("な")]

,title,context,question,answer_start,answer_text,document_id,languages
1466,산리즈카 투쟁,"나리타공항 건설에서 화근이 된 일본 정부의 정책과 그로 인해 발생한 산리즈카 투쟁은 양측에 모두 비참하기만 한 결과를 가져왔다. 이후 공공사업을 둘러싸고 분쟁이 일어나는 현장에서는 “합의형성의 노력을 하지 않은 채 힘에 의지해 사업을 진행하면 힘에 의한 저항을 낳는다”는 자숙을담아 “나리타처럼 되지 말자(成田のようにならないようにしよう )”가 표어가 되었다. \n\n내륙에 공항을 건설하면 토지취득 및 소음 문제가 현저하게 발생할 수밖에 없는데, 나리타에서는 산리즈카 투쟁까지 더해져 막대한 손실을 초래했다. 이 교훈으로 이후 일본은 해상 및 원격지에 공항을 건설하는 경향이 커졌고, 이는 일본 공항의 편리성 저하와 비용 증가로 이어졌다.group=주해|그러나 해상에 건설된 간사이 국제공항에도 중핵파가 공항건설공사의 견학선에 방화하는 테러사건(요코하마요트 소형여객선 폭파사건)을 일으킨 바 있다. group=주해|간사이공항 인공섬 건설 총 공사비가 1조 엔임을 처음 들은 세계 공항관계자는 이 막대한 건설비에 통역의 오역이 아닌지 반문하기도 했다. 또한 토지수용과 행정대집행도 신중하게 실시되도록 되었으니, 산리즈카 투쟁은 일본 공공사업 전반에 엄청난 영향을 미친 것이다 \n\n나리타 하늘과 대지의 역사관은 국가공무원 종합직 초임자와 나리타공항회사 신입사원 연수에 포함되어 있다. 제2의 나리타를 만들지 않도록 산리즈카 투쟁의 교훈을 살리기 위한 노력이라 한다 \n\n한편 개발에 반대하는 주민 측에서도 실력투쟁에서 주민투표 등으로 투쟁방식을 바꾸는 전환점이 되었다 특히 장기간에 걸친 투쟁을 거쳐 이루어진 나리타 공항 문제 심포지엄・나리타 공항 문제 원탁회의는 이후 공공사업 실시 시의 모델 케이스가 되어, 나가라강 하구둑 문제도 원탁회의 방식이 채택되었고, 얀바댐 반대운동을 하던 주민들이 공부하러 나리타 심포지엄을 방문하는 등 일본 전국 각지에서 행해지던 주민운동의 참고사례가 되었다. \n\n관제탑 점거사건 등은 일본 국외에도 크게 보도되어 산리즈카 투쟁의 모습은 영화나 음악 등 예술작품의 테마로 사용되었고, 공공사업의 실패사례로서 큰 주목을 받았다. 예컨대 서독에서는 나리타와 같은 시기에 계획된 뮌헨의 공항건설에 있어서 나리타 공항 문제에 대한 철저한 연구분석을 선행했다. 1969년 공항건설을 결정한 바이에른주 정부는 20년간 259회에 이르는 공청회를 개최하여 반대파를 설득했고, 공항계획이 일부 축소되기는 했지만 최종적으로 착공 5년 후인 1992년 5월 프란츠 요제프 슈트라우스 공항이 개항되어 오늘날 유럽 공항의 일각을 이루고 있다.",공공사업과 관련한 갈등이 유발되는 현장에서 사용되는 문구는 무엇인가?,158,“나리타처럼 되지 말자(成田のようにならないようにしよう )”,38632,"Korean, English, Japanese, Chinese"


In [73]:
# 해당 언어거 어떤식으로 포함되어 있는지 확인
train_df[train_df["context"].str.contains("الكاشغري")]

,title,context,question,answer_start,answer_text,document_id,languages
2869,마흐무드 알카슈가리,"마흐무드 알카슈가리는, 그의 니스바 알카슈가리(الكاشغري)에서 알 수 있듯, 카슈가르에서 태어난 것으로 보인다. 태어난 시기는 1029년 경 또는 1038년 경이라는 것이 일반적인 설명이나, 990년으로 추측하는 학자도 있다. 그의 가문은 카라한 칸국 통치왕가에 속했던 것으로 보인다. 그러나 막상 마흐무드 자신은 이식쿨 인근의 바르스간에 애착을 보였는데, 이는 그의 아버지 후세인 이븐 무함마드 차으르 테긴이 언제인가 이 지역을 통치했기 때문이었다. 반면 고향인 카슈가리에 대해서는 그 주민들이 ‘최악의 인간’이라며 혹평을 쏟아내었는데, 이는 아마 그의 아버지가 카슈가리에서 정치적으로 어려움을 겪었던 것과 연관이 있을 것이다. \n\n알카슈가리의 학문 여정은 당대 지식인 호자 사이푸딘의 딸이었던 어머니, 뷔비 라비야의 교육에서 시작되었다. 1056년부터 1057년까지 카라한 왕조 내부에서 발생한 분란에 휩싸인 그는 고향을 떠나게 되었으나, 부하라와 이란, 이라크 등지를 돌아다니면서도 공부에 여념이 없었다. 그는 아랍어나 페르시아어에도 유창했다. 한편 저작들에서 보이는 다양한 튀르크어 사용 유목민들에 대한 서술로 볼때, 망명기간에 다양한 튀르크어 사용 유목민들이 거주하는 지역 각지를 여행하기도 했던 것으로 보인다\n\n이후 10년 간의 망명 생활의 끝에 바그다드로 이주하여 1072년부터 약 5년 간 튀르크어·아랍어 사전의 집필 작업에 돌입했다. 그 결과가 《튀르크어 대사전》으로, 마흐무드 알카슈가리는 이 책을 압바스 왕조의 칼리프 알무크타디에게 헌정했다\n\n신장 지역에서 발견된 문서들에 따르면 마흐무드 알카슈가리는 만년에 고향으로 돌아갔다. 이슬람력 477년, 즉 서기 1085년 ~ 1086년 사이에 죽었다고 전해진다. 알카슈가리의 무덤은 1981년 카슈가르에서 서쪽으로 45km 떨어진 오팔이라는 곳에서 발견되었다.",마흐무드 알카슈가리가 <튀르크어 대사전>을 만들기 시작한 년도는?,666,1072,52603,"Korean, English, Arabic"


## 년도, 해 로 끝나는 질문에 대한 답변
- 년도나 해를 묻는 답변에서는 년으로 끝나는게 맞아 보이지만 몇개는 `년`을 붙이지 않은 것이 몇개 인지 확인

In [123]:
years_df = train_df[train_df["question"].str.contains(r'년도|연도|해$', regex=True)]
print(len(years_df))
years_df.head()

92


,title,context,question,answer_start,answer_text,document_id,languages
13,장면,"장면은 귀국전 국무총리에 임명되었으나 이듬해인 1951년 2월에 귀국한 후 정식으로 취임하였다. 귀국 무렵 대한민국 국회와 이승만은 갈등하고 있었다. 그는 중간에서 양자의 조정 역할을 하려 했으나 실패하였다 그는 귀국 즉시 총리직에 취임하지 않고, 1주일 간의 여유를 얻어 요인들을 만나 의논해 보았다 그는 “일반적으로 이 박사에 대한 평이 좋지 않아 총리직을 맡을 생각이 간절하지 않았으나 이왕 인준도 받았으니 하는 데까지 하다가 할 수 없으면 그만두라”라는 주변의 의견대로 총리직을 맡게 되었다 당초 이승만은 장면에게 인사권을 위임하겠다고 하였으나, “5석 중 3석을 총리가 정하였으니 내무에는 이순용(李淳鎔), 국방에는 이기붕을 써 주시오”하고 종용하자, 당초 약속과는 달랐으나 장면은 받아들였다 국무총리 재임 중 그는 호화로운 식단을 기피하고 직접 도시락을 싸들고 출퇴근하였다.\n\n \n8월에는 자유당이 창당되자 자유당에 입당했다. 12월에는 원외 자유당이 결성되면서 장면이 속한 자유당은 원내 자유당으로 불리게 되었다. 원내 자유당에서는 1952년 제2대 정부통령 선거에서 그를 대통령 후보로 추대하려는 일파와, 내각 책임제로 개헌하여 대통령직을 유명무실화하게 한 뒤 그를 내각 책임제 하의 국무총리로 추대하려는 세력이 나타났다. 흥사단도 장면을 지지하였다. 이후 흥사단의 지지를 이유로 장면은 흥사단계로 분류되었으나 장면은 이를 부정하였다. 장면을 대통령 후보로 추대하려는 세력과 내각책임제 개헌 후 장면을 총리로 추대하려는 세력의 존재가 이승만의 귀에 들어가면서 장면은 정치적으로 궁지에 몰리게 된다.",장면이 한국으로 돌아온 년도는?,26,1951년,6930,"Korean, English, Chinese"
16,제1차 세계 대전,"양측 모두 2년 동안 서로에게 결정적인 타격을 줄 수 있는 공격을 하지 못했다. 1915~1917년 동안, 대영제국 및 프랑스는 전략, 전술적 방향의 측면의 선택 때문에 독일보다 더 많은 사상자로 고통받았다. 독일은 오직 하나의 주요 공세만 시도했지만 연합군은 독일의 방어선을 돌파하기 위한 여러 시도를 하였다.\n\n1916년 2월 독일은 프랑스의 베르됭에서 프랑스 진지를 공격하면서 베르됭 전투가 시작되었다. 1916년 12월 프랑스의 반격으로 공세 시작 지점 이전까지 독일군을 퇴각시키기 전까지 전투는 독일군이 많은 이익을 보았다. 프랑스군의 사상자는 엄청났지만, 독일군의 사상자 또한 이에 못지 않게 높아서 양 측 교전국의 사상자는 70만명 에서 97만 5천명 가까이 된다. 베르됭은 프랑스의 의지 및 희생의 상징이 되었다. \n\n1916년 7월부터 11월까지 영국-프랑스군은 솜 전투를 통해 독일군을 공격했다. 이 공격이 시작한 7월 1일 당시 영국 육군은 역사상 가장 큰 사상자가 발생해, 첫날 전투에만 19,240명이 사망하는 등 총 사상자가 75,470명이나 되었다. 솜 공세 전체 기간 동안 영국군의 사상자는 약 42만명이었다. 프랑스군 또한 대략 20만명의 사상자가 나왔으며, 독일군은 50만명의 사상자가 나왔다. \n\n1916년 전체 기간 동안 베르됭에서의 오래된 공격은 솜 전투의 심각한 사상자와 합쳐져 프랑스군이 지쳐 붕괴 위기까지 달했다. 1917년 4월 5일 니벨 공세가 연합국의 실패로 끝난 이후 영국 및 프랑스군의 헛된 정면 공격 시도는 프랑스 병사(poilu)의 프랑스군 반란으로 이어졌다. 동시에 일어난 영국의 아라스 전투는 제한된 범위에서 이루어져 궁극적으로 전략적 가치는 떨어졌지만 더 많은 성공을 거두었다. 아라스 공세의 일부분으로 캐나다 군단이 비미 리지를 점령했고, 이는 해당 국가에게 매우 중요하게 되었다. 캐나다의 국가정체성이 전투로 탄생되었다는 생각은 이후 캐나다의 군사 및 일반 사학계에서 널리 퍼지게 되는 의견이 되었다. \n\n1917년 7월부터 11월까지 이루어진 파스샹달 전투는 이 기간 벌어진 영국의 마지막 대규모 공세(프랑스의 지원)가 되었다. 이 공세는 10월 전장이 진흙탕으로 되기 전까지 연합국 사이의 약속으로 진행되었다. 사상자에 대해서는 많은 의견이 존재하지만 거의 20만명에서 40만명 사이로 추측한다.\n\n서부 전선의 참호전은 몇 년 동안 많은 지역의 점령이 이루어지지 않게 되었고, 그 결과 이 전선은 종종 정적이고 변하지 않는 전선이라고 생각한다. 그러나, 이 기간 동안 영국, 프랑스, 독일은 끊임없는 전술의 발전으로 인해 새로운 전장으로 나타났다.",영국군 약 42만명의 사상자가 발생한 전투가 시작된 연도는?,414,1916년,5510,"Korean, English"
22,호르스트 슈만,"그러다가 1944년에 아우슈비츠를 떠나 작센의 군 병원으로 이동한 슈만은 서부전선에서 군의관으로 일하다가 1945년 1월에 멩겔레와 비슷하게 미군의 포로가 되었다. 그러나 독일의 패전 후인 1945년 10월에 호르스트 슈만은 석방되었고,1946년에 그라트벡의 스포츠 관련 의사로 일하면서 전쟁이 끝난 후 한동안 별다른 일 없이 평안하게 지냈다, 그러나 1951년에 사냥 엽총면허증을 발급받으려고 제출한 신청서의 심사 과정에서 경찰 당국은 그가 수배 인물임을 파악하였고 체포영장이 발부되자 낌새를 느낀 그는 여권도 없이 곧바로 해외로 달아났다.\n\n호르스트 슈만은 처음에는 미국을 전전했다 호르스트 슈만이 한동안 어디에 있는지 파악이 되지 않았으나 우연한 기회에 밝혀지게 되었다. 독일에서 발간되는 ""그리스도와 세계""라는 잡지에 아프리카 가나의 오지에서 의학기술을 베푸는 '제2의 슈바이처 박사'라는 기사가 사진과 함께 소개된 것이 결정적이였다. 호르스트 슈만은 가나의 국가수반인 콰메 은크루마가 이끄는 은크루마 정권이 자신을 끝까지 지켜줄 것이라고 믿었지만 1966년 은크루마 정권이 무너지게 되자 곧바로 독일로 이송되었다. 결국 마지막엔 아프리카의 가나까지 달아나기도 했지만 결국 1966년에 가나에서 붙잡혀서 서독정부에 인계되었던 것이다.\n\n1970년 9월에 슈만의 재판이 시작되었다. 슈만은 잡히기 이전부터 이미 나치 독일의 강제수용소에서 1만 5천여명을 학살한 혐의가 있었던 강제수용소 의사로 재판을 받았는데, 호르스트 슈만은 24일 죽음의 감방에서 치명적인 일산화탄소 가스를 집어넣는 버튼을 자기가 눌렀었다고 자백했으며 그는 1만 4천 5백명의 장애인과 정신질환자인 독일 사람과 약 8백여명의 전쟁포로를 학살한 사실을 인정했다, 그러나 6년 간의 수감 생활 끝에 법원은 1972년 7월 29일에 건강상의 이유를 들어 호르스트 슈만을 형무소에서 풀려나게 했다, 의사들의 진단서에는 그의 병세가 매우 위중하다고 기록되어 있지만 그는 11년이 넘는 시간 동안 살았다고 한다. 그러다가 11년 후인 1983년 5월 5일 호르스트 슈만은 77세라는 나이로 천수를 누리고 사망했다.",슈만이 미군에게 포로로 잡혔던 년도는?,59,1945년,51125,"Korean, English"
32,외국의 시리아 내전 관여,"외국의 시리아 내전 관여는 2011년 3월 시작된 시리아 내전에 참여한 세력에 정치, 군사, 작전적인 지원을 하는 것과 외국의 활발한 개입을 의미한다. 시리아 내전에 참여한 대부분의 세력은 외국으로부터 다양한 종류의 지원을 받았다. 시리아 내전은 미국과 러시아, 그리고 이란과 사우디아라비아를 중심으로 한 역내 세력과 세계 강대국의 대리 전쟁이 중복되는 것으로 묘사되고 있다.. \n\n시리아 바트당 정부는 이란과 러시아에 군사 및 정치적인 지원을 받고 있으며, 레바논의 헤즈볼라와 시리아에 본부를 둔 팔레스타인 무장단체 PFLP-GC도 시리아 정부를 돕고 있다. 공개적이고 합법적으로 시리아에 기지를 보유한 유일한 해외 국가인 러시아는 시리아 정부의 요청에 따라 2015년 9월 30일부터 ISIL과 다른 반정부 단체에 맞서 공중전을 시작했다. 러시아의 군사 행위는 미국과 터키의 비난을 받았다. 터키는 2015년 11월부터 러시아 전투기가 영공을 침범했다는 이유로 요격하기도 했으며, 터키가 지원하는 반정부군에 러시아의 공격이 발생하자 이를 비판하기도 

In [126]:
# 년도 or 연도에 끝에 오는 질문의 답변 둘러보기
for years in years_df["answer_text"]:
  print(years)

1951년
1916년
1945년
2015년
1088년
1931년
1966년
1665년
1856년
1994년
1610년
1966년
2006년
1897년
1636년
2008년
1962년
1807년
1936년
1309년
1603년
1053
293년
1990년
1945년
2014
2004년
2008년
2008년
2015년
1992년
1014년
2004년
1826년
1951년
1801
1944년
1990년
1939년
1918년
2011년
1619년
1931년
1059년
1999년
1433년(세종 15년)
551년
2008년
1948년
1938년
1927년
2007년
2006년
2004년
1978년
2005년
1207년
1681년
미카와 국 다이주 사
1867년
2008년
2014년
1934년
824년
2006년
2005년
1946년
1560년
기원전 331년
1997년
893년
1928년
1920년
1755년
1515년
독일의 미드필더 토니 크로스
1072
1119년
2016년
1879년
1860년
1945년
1896년
1974년
2008년
1999년
1793
1945년
1502년
1362년
1949년
1978년


In [124]:
len(years_df[years_df["answer_text"].str.contains("년")])

85

# wiki 데이터 context 데이터 탐색


In [41]:
import json

In [42]:
# wiki 데이터 불러오기
# 데이터셋 불러오기
path = "/content/drive/MyDrive/네이버 부스트 캠프/프로젝트/Level2_프로젝트/MRC/data/wikipedia_documents.json"
with open(path, 'r') as f:
  json_data = json.load(f)

wiki_json_df = pd.DataFrame(json_data)
wiki_json_df = wiki_json_df.transpose()
wiki_json_df = wiki_json_df[['title', 'text', 'document_id']]
wiki_json_df.head()

title  \
0    나라 목록   
1    나라 목록   
2      백남준   
3  아오조라 문고   
4  아오조라 문고   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

In [ ]:
# 랜덤 샘플링한 데이터
# 전체 데이터 중 200개를 랜덤으로 뽑기
sample_context = random.sample(list(wiki_json_df["text"]), 200)

print(sample_context)

In [ ]:
for context in sample_context:
  print(context)
  print("=" * 50)

In [45]:
word_count = wiki_json_df['text'].str.split().str.len().sum()
newline_count = wiki_json_df['text'].str.count('\n').sum()
print(f"개행이 포함되어 있는 데이터의 수 : {newline_count} / {word_count}")

개행이 포함되어 있는 데이터의 수 : 370078 / 9791882


In [46]:
len(wiki_json_df["text"])

60613

In [47]:
newline_counts = np.array(get_newline(wiki_json_df["text"]))
print(np.sum(newline_counts[:,1],axis=0))

29106


In [48]:
# 특정 칼럼을 기준으로 중복된 값을 Counter 함수로 찾은 뒤 리턴하는 함수
def wiki_count_duplication(column):
  count = dict(Counter(wiki_json_df[column]))
  count = sorted(count.items(),
                 reverse=True,
                 key=lambda item:item[1])

  return count

In [49]:
def wiki_print_duplication(column):
  total = len(wiki_json_df)
  unique_column = len(wiki_json_df[column].unique())

  print(f"전체 데이터 개수 : {total}")
  print(f"unique한 {column} 개수 : {unique_column}")

  if total != unique_column:
    print(f"중복된 값이 있습니다. 중복된 값 개수 : {total - unique_column}")
  else:
    print("중복된 값이 없습니다.")

In [ ]:
wiki_print_duplication("document_id")
document_count = wiki_count_duplication("document_id")
print(document_count)